<a href="https://colab.research.google.com/github/nestorbravo/Fractal/blob/main/Higher_regularity_for_quantum_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
import itertools
from itertools import combinations
from IPython.display import display, Math
import cmath
import sys

# 1. Definition of a working Algebra Class

## 1.1 Class Algebra

The following Python class **Algebra** provides a computational framework for performing operations on finite-dimensional C*-algebras in the context of quantum graphs. In this setting, the algebra $A$ is typically decomposed as a direct sum of matrix blocks $A \simeq \bigoplus_i M_{n_i}(\mathbb{C})$, and the class allows us to construct and manipulate the key algebraic structures associated with $A$.



In [ ]:
class Algebra:
    def __init__(self, coef, trace):
        # coef: list of matrix block sizes in the decomposition A ≅ ⊕ M_n(C)
        # trace: this is the positive functional to realize algebra A as hilbert Space, usually the normaized trace
        self.coef = coef
        self.m = []         # multiplication matrix m: A⊗A → A
        self.mt = []        # transpose of m, m^t: A → A⊗A
        self.trace = []     # trace operator Tr: A → C
        self.counit = []    # counit ε associated to the trace
        self.get_mult_mat() # initialize multiplication matrix and its transpose
        self.get_trace(trace) # initialize trace and counit

    def dim(self):
        # Returns the vector space dimension of A
        # If A ≅ ⊕_i M_{n_i}(C), dimension is ∑ n_i^2
        d = 0
        for i in range(len(self.coef)):
            d += self.coef[i]**2
        return d

    def say_hi(self):
        # Print basic information about the algebra
        print("Algebra \n")
        name = "$A = "
        for i in range(len(self.coef)):
            # Build the expression A = ⊕ M_{n_i}(C)
            name += f"M_{self.coef[i]} (\mathbb C)"
            if i != len(self.coef) - 1:
                name += "\oplus "

        name += "$"
        display(Math(name))

        print(f"Dimension = {self.dim()}" )
        print("State operator: ")
        print(f"tr = {self.trace}")
        print("Multiplication operator: ")
        display(Math("$m: C(X)\otimes C(X) \Rightarrow C(X)$"))
        display(Math("$mm^t =$"))
        # Display m m^t, which encodes δ^2 in planar algebra formalism
        print(self.m.dot(self.mt).toarray())

    def get_mult_mat(self):
        # Constructs the multiplication matrix m: A⊗A → A
        coords, vals, shape = get_mult_mat_coords(self.coef)

        rows = coords[:, 0]
        cols = coords[:, 1]
        data = vals

        m = coo_matrix((data, (rows, cols)), shape=shape)

        self.m = m
        self.mt = m.T # transpose of m

    def get_trace(self, Tr):
        # Construct the trace operator Tr: A → C
        self.trace = Tr(self.coef)
        # The counit ε: A → C as a column vector
        self.counit = self.trace.reshape(-1, 1)

    def SchurProd(self, A, B):
        # Computes the Schur (Hadamard) product in the algebra
        K = np.kron(A, B)                # tensor product
        P = np.matmul(K, self.mt.toarray())
        SP = np.matmul(self.m.toarray(), P)
        return SP

    def Fourier(self, T):
        # Fourier transform associated to the algebra (operator A→A)
        F1 = np.kron(A.counit, np.kron(A.counit, np.identity(A.dim())))
        F2 = np.kron(A.mt.toarray(), np.kron(np.identity(A.dim()), np.identity(A.dim())))
        F3 = np.kron(np.identity(A.dim()), np.kron(T, np.identity(A.dim())))
        F4 = np.kron(np.identity(A.dim()), np.kron(np.identity(A.dim()), A.m.toarray()))
        F5 = np.kron(np.identity(A.dim()), np.kron(A.trace, A.trace))

        # Compose all five operators
        return np.matmul(F5, np.matmul(F4, np.matmul(F3, np.matmul(F2, F1))))

    def FourierInv(self, T):
        # Inverse Fourier transform
        F1 = np.kron(A.m.toarray(), np.identity(A.dim()))
        F2 = np.kron(np.identity(A.dim()), np.kron(T, np.identity(A.dim())))
        F3 = np.kron(np.identity(A.dim()), A.mt.toarray())
        return np.matmul(F1, np.matmul(F2, F3))

    def Laplacian(self, T):
        # Laplacian associated with operator T (generalizes graph Laplacian)
        L1 = np.kron(T, np.identity(self.dim()))
        L2 = np.kron(self.trace, np.identity(self.dim()))
        delta2 = self.trace.dot(self.counit)[0] # δ^2 constant
        return delta2**(-1/2) *  np.matmul(L2, np.matmul(L1, self.mt.toarray())) - T

    def CompleteGraph(self):
        # Returns the operator representing the complete graph (all vertices connected)
        eta = self.counit
        return self.dim()**(3/2) * eta.dot(eta.T)

    def TrConjugate(self, T):
        # Conjugation with respect to the trace: adjoint in the inner product induced by Tr
        C = T
        for i in range(len(self.coef)):
            for j in range(self.coef[i]):
                for k in range(self.coef[i]):
                    C[j][k] = np.conjugate(T[k][j]) # conjugate transpose block-wise
        return C

<>:27: SyntaxWarning: invalid escape sequence '\m'
<>:29: SyntaxWarning: invalid escape sequence '\o'
<>:38: SyntaxWarning: invalid escape sequence '\o'
<>:27: SyntaxWarning: invalid escape sequence '\m'
<>:29: SyntaxWarning: invalid escape sequence '\o'
<>:38: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipython-input-2134291785.py:27: SyntaxWarning: invalid escape sequence '\m'
  name += f"M_{self.coef[i]} (\mathbb C)"
/tmp/ipython-input-2134291785.py:29: SyntaxWarning: invalid escape sequence '\o'
  name += "\oplus "
/tmp/ipython-input-2134291785.py:38: SyntaxWarning: invalid escape sequence '\o'
  display(Math("$m: C(X)\otimes C(X) \Rightarrow C(X)$"))


In fact, one can verify that our formula for Schur product with tracial state is the following:
$$
(\hat{T} \star \hat{S})_k^\ell
= \sum_{i,j,m,n} m_{mn}^\ell \, (\hat{T} \otimes \hat{S})_{(m,n),(i,j)} \, m_{ij}^k,
$$
Here, $m$ is the multiplication matrix.

## 1.2 Useful combinatorial functions for Algebra Class


This is just a miscelanea of combinatorial functions, useful for the definition of algebra class

In [ ]:
def mult_coords(dim):
    """
    Generate the index pairs corresponding to the multiplication table of a single matrix block M_dim(C).

    In a matrix algebra M_dim(C), the product of basis elements E_ij * E_kl is nonzero
    if and only if j == k. This function constructs a list of coordinate pairs [i,j] such that
    multiplying the basis element indexed by B2[j] with B[i] gives a valid output index in the flattened algebra.
    """
    coords = []

    B = [[a, b] for a in range(dim) for b in range(dim)]  # Basis indices (i,j) of M_dim(C)
    B2 = [[x, y] for x in B for y in B]  # All pairs of basis elements for tensor product

    for j in range(len(B2)):
        for i in range(len(B)):
            if B2[j][0][1] == B2[j][1][0]:  # Ensure middle indices match for multiplication
                if B2[j][0][0] == B[i][0]:  # Row index matches
                    if B2[j][1][1] == B[i][1]:  # Column index matches
                        coords.append([i, j])

    return coords


def get_mult_mat_coords(coef):
    """
    Compute the coordinates and values needed to construct the multiplication matrix of a
    direct sum algebra A = ⊕_i M_{n_i}(C).

    The function returns:
      - coords: a list of [row_index, col_index] for the nonzero entries in the flattened multiplication matrix
      - vals: the corresponding nonzero values (normalized by the δ-form)
      - shape: the shape of the multiplication matrix (dim, dim^2)

"""
    block_coords = []
    sums = len(coef)
    dim = int(np.sum([n**2 for n in coef]))  # Total dimension of the algebra

    for i in range(len(coef)):
        block_coords.append(mult_coords(coef[i]))  # Get coordinates for each block

    It = [[a, b] for a in range(sums) for b in range(sums)]  # Pairs of blocks

    coords = []
    vals = []

    for i in range(len(coef)):
        I = It.index([i, i])  # Diagonal block
        c0 = int(np.sum([coef[It[s][0]] * coef[It[s][1]] for s in range(I)]))  # Column offset
        r0 = int(np.sum([coef[s]**2 for s in range(i)]))  # Row offset
        for j in range(len(block_coords[i])):
            coords.append([block_coords[i][j][0] + r0, block_coords[i][j][1] + c0])
            vals.append(dim**(1/2) / coef[i]**(1/2) )  # Normalization factor

    return np.array(coords), np.array(vals), (dim, dim**2)


def kdelta(i,j):
  if i == j:
    return 1
  return 0

def verify_diag(A):
    # Checks if a matrix is diagonal with all diagonal elements equal
    # Returns (True, diagonal_value) if so, else (False, None)
    a = A[0, 0]

    if not np.allclose(np.diag(A), a):
        return False, None

    if not np.allclose(A, a * np.eye(A.shape[0])):
        return False, None

    return True, a

def Rotate(matrix, n):
    # Rotates an n^2 x n^2 matrix by flattening each n x n block into columns
    # Returns the rotated matrix
    N = n**2
    result = np.zeros((N, N))

    col_idx = 0
    for i in range(0, N, n):
        for j in range(0, N, n):
            block = matrix[i:i+n, j:j+n]
            result[:, col_idx] = block.flatten(order='C')
            col_idx += 1

    return result

## 1.3 Definition of state

In this section, user may define your positive functional $\psi$ for the $C^*$-álgebra $A$, in order to get a faithful state $(a,b)\mapsto \psi(ab^*)$

In [ ]:
# This is the normalized trace tr(T)/dim(A). The state can be defined arbitrarily. All calculations in this work are based on this state.

def trace_norm(coef):
  Tr = []

  dim = int(np.sum([n**2 for n in coef]))

  for i in range(len(coef)):
    Tri = []
    B = [[a,b] for a in range(coef[i]) for b in range(coef[i])]

    for b in B:
      if b[0] == b[1]:
        Tri.append(coef[i])
      else:
        Tri.append(0)

    Tr += Tri

  return np.array(Tr)/dim

# 2. Verifiers

## 2.1 Schur idempotency verifier

The following function verifies if some operator $T: A\to A$ is a Schur idempotent with the renormalization $\frac{1}{\dim(A)}$.

In [ ]:
def verifier(A, T):
  delta = A.trace.dot(A.counit)[0]
  print("In Algebra A = {}".format(A.coef) )
  if np.allclose(T, 1/A.dim() * A.SchurProd(T,T)):
    print("T is Schur Idempotent:\n")
    print(T)
  else:
    print("T is not Schur Idempotent\n ")
    print("T * T = ")
    print(1/A.dim() * A.SchurProd(T,T))
    return None

  L = A.Laplacian(T)
  print("Laplacian:")
  print(np.round(L, 4))

  print("Laplacian Eigenvalues:")
  evals, evec = np.linalg.eig(L)
  print(np.round(evals, 4))

  deg = L + T

  print("Degree Matrix:")
  print(deg)

  print("Laplacian Rank:")
  print(np.linalg.matrix_rank(L))

  print("Trace/2:")
  print(L.trace()/2)

  return None

## 2.2 Higher Regularity Verifiers

The following function determines if a graph $T$ is regular, connected and even if it is a tree, a forest or none of them.

In [ ]:
def regularity(A,T):
  L = A.Laplacian(T)
  isDiag, deg = verify_diag(L + T)
  rank = np.linalg.matrix_rank(L)

  isConn = False
  if A.dim() - rank == 1:
      isConn = True

  if isDiag:
    print("Is regular, degree = {}".format(deg))

  if isConn:
    print("Is connected")
  else:
    print("Has {} connected components".format(A.dim() - rank))

  print("Trace of Lap = {}".format(L.trace()))

  print("Rank of Lap = {}".format(rank))

  if L.trace() == rank:
    if isConn:
      print("Is a Tree")
    else:
      print("Is a Forest")

  return isDiag, isConn, deg

The following function check if a graph $T$ is 2-regular and compute its parameters.

In [ ]:
def IrreGraph(A, T):
    # Make a quantum graph irreflexive.
    # A is the algebra object, T is the adjacency operator of the graph
    tol = 1e-14  # tolerance for numerical comparisons
    id = np.identity(A.dim())

    # Check if T has self-loops (non-zero diagonal in Schur product with identity)
    if np.abs(A.SchurProd(id, T)[0][0]) > tol:
        T = T - id  # Remove self-loops to make graph irreflexive
    else:
        pass

    return T

def two_Gromada(A, T, comp, delta_2):
    # Compute the 2-point regularity parameters (k, lambda, mu) for a quantum graph
    # A: algebra object, T: adjacency operator, comp: complete graph, delta_2: delta^2
    T = IrreGraph(A, T)  # Ensure the graph is irreflexive
    tamanio = T.shape[0]  # dimension of adjacency matrix
    tol = 1e-14  # numerical tolerance
    id = np.identity(A.dim())

    landa = 0
    mu = 0
    k = 0

    # Compute squares and differences needed for 2-regularity
    T_2 = np.matmul(T, T)  # matrix square of T
    Q = comp - T - id  # complement of T (excluding identity)

    # Schur products for computing parameters
    T_T2 = A.SchurProd(T, T_2)
    Q_T2 = A.SchurProd(Q, T_2)
    id_T2 = A.SchurProd(id, T_2)

    # Compute parameter k from diagonal of id_T2
    k = id_T2[0][0] / (delta_2 * id[0][0])

    # Loop over T and Q to determine lambda and mu
    i = 0
    while i < 2:
        if i == 0:
            Ope = T
            OpeConv = T_T2
        else:
            Ope = Q
            OpeConv = Q_T2

        Salir = False  # flag to exit nested loops once first non-zero element is found
        for fila in range(tamanio):
            if Salir:
                break
            for col in range(tamanio):
                if np.abs(Ope[fila][col]) > tol:
                    Salir = True
                    if i == 0:
                        landa = OpeConv[fila][col] / (delta_2 * Ope[fila][col])
                    else:
                        mu = OpeConv[fila][col] / (delta_2 * Ope[fila][col])
                    break
        i += 1

    # Check if the 2-point regularity equation holds
    error = np.linalg.norm(T_2 - k * id - landa * T - mu * Q)
    print(error)
    if error < tol:
        print(f"The matrix is 2-regular. Their parameters are: k={k}, lambda={landa}, mu={mu}.")
    else:
        print("The matrix isn't 2-regular.")

    return landa, mu, k

The following functions are modules of functions *Parameters3PR* and *Check3PR*. The first one compute an aproximation of the parameters $q_0, q_1, q_2$ and $q_3$ for a graph $T$; if this function fails computing a parameter, it is suficient to conclude $T$ isn't 3-point regular. The second one determines if a graph $T$ is $3$-point regular using parameters specified by the user.

In [ ]:
# Compute auxiliary operators for q0 and q3 parameters in 3-point regularity.
def AuxiliarQ0Q3(A, T, comp, delta_2):
    T = IrreGraph(A, T)  # Ensure T is irreflexive
    i = 0
    T_F = A.FourierInv(T)  # Inverse Fourier of T
    id = np.identity(A.dim())

    while i <= 1:
        if i == 0:
            OpeRand_F = T_F
        else:
            OpeRand_F = A.FourierInv(comp - id - T)  # Inverse Fourier of complement

        # Build the auxiliary operator using multiple Kronecker products and multiplications
        B1 = A.mt.toarray()
        B2 = np.kron(A.mt.toarray(), id)
        B3 = np.kron(id, np.kron(np.matmul(A.mt.toarray(), T), id))
        B4 = np.kron(id, np.kron(T, np.kron(T, id)))
        B5 = np.kron(OpeRand_F, OpeRand_F)
        B6 = np.kron(A.trace, np.kron(OpeRand_F, A.trace))

        Ope = np.matmul(B6, B5)
        Ope = np.matmul(Ope, B4)
        Ope = np.matmul(Ope, B3)
        Ope = np.matmul(Ope, B2)
        Ope = np.matmul(Ope, B1) / np.sqrt(delta_2**3)  # Normalize

        if i == 0:
            OpeL0 = Ope
        else:
            OpeL3 = Ope
        i += 1

    return OpeL0, OpeL3


# Compute auxiliary operators for q1 and q2 parameters.
def AuxiliarQ1Q2(A, T, comp, delta_2):
    T = IrreGraph(A, T)
    i = 0
    id = np.identity(A.dim())

    T_F = A.FourierInv(T)
    Tc_F = A.FourierInv(comp - id - T)
    id_F = A.FourierInv(id)

    while i <= 1:
        if i == 0:  # q1
            OpeRand_F1 = T_F
            OpeRand_F2 = T_F
            OpeRand_F3 = Tc_F
        else:  # q2
            OpeRand_F1 = Tc_F
            OpeRand_F2 = Tc_F
            OpeRand_F3 = T_F

        # Build the auxiliary operator
        B1 = A.mt.toarray()
        B2 = np.kron(A.mt.toarray(), id)
        B3 = np.kron(id, np.kron(np.matmul(A.mt.toarray(), T), id))
        B4 = np.kron(id, np.kron(T, np.kron(T, id)))
        B5 = np.kron(OpeRand_F1, OpeRand_F2)
        B6 = np.kron(A.trace, np.kron(OpeRand_F3, A.trace))

        Ope = np.matmul(B6, B5)
        Ope = np.matmul(Ope, B4)
        Ope = np.matmul(Ope, B3)
        Ope = np.matmul(Ope, B2)
        Ope = np.matmul(Ope, B1)
        Ope = Ope / np.sqrt(delta_2**3)  # Normalize

        if i == 0:
            OpeL1 = Ope
        else:
            OpeL2 = Ope
        i += 1

    return OpeL1, OpeL2


# Compute triangle operators for q0 and q3 (or all if cond=False)
def TrianglesKQ0Q3(A, T, comp, delta_2, cond):
    T = IrreGraph(A, T)
    id = np.identity(A.dim())
    Tc = comp - id - T
    i = 0

    if cond:  # Compute only for q0 and q3
        while i < 2:
            OpeRand = T if i == 0 else Tc
            # Construct the triangle operator using Kronecker products
            R_1 = np.matmul(np.kron(OpeRand, OpeRand), A.mt.toarray())
            R_2 = np.kron(A.mt.toarray(), id)
            R_3 = np.kron(id, np.kron(OpeRand, id))
            R_4 = np.kron(id, A.m.toarray())
            OpeT = np.matmul(R_4, R_3)
            OpeT = np.matmul(OpeT, R_2)
            OpeT = np.matmul(OpeT, R_1)

            if i == 0:
                OpeT0 = OpeT
                OpeTk = OpeT  # For technical reasons
            else:
                OpeT3 = OpeT
            i += 1
    else:  # Compute all triangle operators
        while i < 3:
            if i == 0:
                OpeRand = id
            elif i == 1:
                OpeRand = T
            else:
                OpeRand = Tc

            R_1 = np.matmul(np.kron(OpeRand, OpeRand), A.mt.toarray())
            R_2 = np.kron(A.mt.toarray(), id)
            R_3 = np.kron(id, np.kron(OpeRand, id))
            R_4 = np.kron(id, A.m.toarray())
            OpeT = np.matmul(R_4, R_3)
            OpeT = np.matmul(OpeT, R_2)
            OpeT = np.matmul(OpeT, R_1)

            if i == 0:
                OpeTk = OpeT
            elif i == 1:
                OpeT0 = OpeT
            else:
                OpeT3 = OpeT
            i += 1

    return OpeTk, OpeT0, OpeT3


# Compute triangle operators for lambda, mu, q1, q2
def TrianglesLMuQ1Q2(A, T, comp, delta_2, cond):
    T = IrreGraph(A, T)
    id = np.identity(A.dim())
    Tc = comp - id - T
    j = 0

    if cond:  # Compute only first triangle for q1 and q2
        while j < 2:
            OpeRand1 = T if j == 0 else Tc
            OpeRand2 = Tc if j == 0 else T
            R_1 = np.matmul(np.kron(OpeRand1, OpeRand1), A.mt.toarray())
            R_2 = np.kron(A.mt.toarray(), id)
            R_3 = np.kron(id, np.kron(OpeRand2, id))
            R_4 = np.kron(id, A.m.toarray())
            OpeD = np.matmul(R_4, R_3)
            OpeD = np.matmul(OpeD, R_2)
            OpeD = np.matmul(OpeD, R_1)

            if j == 0:
                TriQ1 = OpeD
                TriL = OpeD
                TriM = OpeD
            else:
                TriQ2 = OpeD
            j += 1
    else:  # Compute all triangle operators (lambda, mu, q1, q2)
        while j < 4:
            if j == 0:  # lambda
                OpeRand1, OpeRand2, OpeRand3 = T, T, id
            elif j == 1:  # mu
                OpeRand1, OpeRand2, OpeRand3 = Tc, Tc, id
            elif j == 2:  # q1
                OpeRand1, OpeRand2, OpeRand3 = T, T, Tc
            else:  # q2
                OpeRand1, OpeRand2, OpeRand3 = Tc, Tc, T

            # Construct triangle operators
            R_1 = np.matmul(np.kron(OpeRand1, OpeRand2), A.mt.toarray())
            R_2 = np.kron(A.mt.toarray(), id)
            R_3 = np.kron(id, np.kron(OpeRand3, id))
            R_4 = np.kron(id, A.m.toarray())
            OpeD1 = np.matmul(R_4, R_3)
            OpeD1 = np.matmul(OpeD1, R_2)
            OpeD1 = np.matmul(OpeD1, R_1)

            R_1 = np.matmul(np.kron(OpeRand2, OpeRand3), A.mt.toarray())
            R_2 = np.kron(A.mt.toarray(), id)
            R_3 = np.kron(id, np.kron(OpeRand1, id))
            R_4 = np.kron(id, A.m.toarray())
            OpeD2 = np.matmul(R_4, R_3)
            OpeD2 = np.matmul(OpeD2, R_2)
            OpeD2 = np.matmul(OpeD2, R_1)

            R_1 = np.matmul(np.kron(OpeRand3, OpeRand1), A.mt.toarray())
            R_2 = np.kron(A.mt.toarray(), id)
            R_3 = np.kron(id, np.kron(OpeRand2, id))
            R_4 = np.kron(id, A.m.toarray())
            OpeD3 = np.matmul(R_4, R_3)
            OpeD3 = np.matmul(OpeD3, R_2)
            OpeD3 = np.matmul(OpeD3, R_1)

            Ope = OpeD1 + OpeD2 + OpeD3

            if j == 0:
                TriL = Ope
            elif j == 1:
                TriM = Ope
            elif j == 2:
                TriQ1 = Ope
            else:
                TriQ2 = Ope
            j += 1

    return TriL, TriM, TriQ1, TriQ2


# Compute scalar parameters q3, q2, q1, q0 (partial computation for 3-regularity)
def Parameters3PR(A, T, comp, delta_2):
    tol = 1e-12
    # Compute triangle operators needed for q parameters
    OpeTk, OpeTq0, OpeTq3 = TrianglesKQ0Q3(A, T, comp, delta_2, True)
    OpeTL, OpeTM, OpeTq1, OpeTq2 = TrianglesLMuQ1Q2(A, T, comp, delta_2, True)

    # Compute auxiliary operators
    OpeLq1, OpeLq2 = AuxiliarQ1Q2(A, T, comp, delta_2)
    OpeLq0, OpeLq3 = AuxiliarQ0Q3(A, T, comp, delta_2)

    tamanioFilas = OpeTq0.shape[0]
    tamanioCol = OpeTq0.shape[1]

    # Loop over the four parameters
    for i in range(4):
        q = 0
        if i == 0:
            OpeL = OpeLq0
            OpeT = OpeTq0
        elif i == 1:
            OpeL = OpeLq1
            OpeT = OpeTq1
        elif i == 2:
            OpeL = OpeLq2
            OpeT = OpeTq2
        else:
            OpeL = OpeLq3
            OpeT = OpeTq3

        salir = False
        for fila in range(tamanioFilas):
            if salir:
                break
            for col in range(tamanioCol):
                if np.abs(OpeT[fila][col]) > tol:
                    salir = True
                    q = OpeL[fila][col] / OpeT[fila][col]
                    break

        if i == 0:
            q_3 = q
        elif i == 1:
            q_2 = q
        elif i == 2:
            q_1 = q
        else:
            q_0 = q

    return q_3, q_2, q_1, q_0


# Check if a quantum graph is 3-regular using provided constants
def Check3PR(A, T, comp, delta_2, k, l, m, q3, q2, q1, q0):
    T = IrreGraph(A, T)
    tol = 1e-10

    OpeTk, OpeTq0, OpeTq3 = TrianglesKQ0Q3(A, T, comp, delta_2, False)
    OpeTL, OpeTM, OpeTq1, OpeTq2 = TrianglesLMuQ1Q2(A, T, comp, delta_2, False)

    # Compute normalized triangle operator
    triangle = delta_2 * np.matmul(np.kron(T, T), np.matmul(A.mt.toarray(), T))

    # Check if the linear combination of operators matches
    error = np.linalg.norm(triangle - (k*OpeTk + l*OpeTL + m*OpeTM + q3*OpeTq0 + q2*OpeTq1 + q1*OpeTq2 + q0*OpeTq3))
    print(error)

    regularity = (error < tol)
    return regularity

# Testing area

# 3. Some examples

The following is an example of how to declare an algebra:

In [ ]:
A = Algebra([2], trace_norm)

A.mt.toarray()

array([[1.41421356, 0.        , 0.        , 0.        ],
       [0.        , 1.41421356, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [1.41421356, 0.        , 0.        , 0.        ],
       [0.        , 1.41421356, 0.        , 0.        ],
       [0.        , 0.        , 1.41421356, 0.        ],
       [0.        , 0.        , 0.        , 1.41421356],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.41421356, 0.        ],
       [0.        , 0.        , 0.        , 1.41421356]])

In [ ]:
T = np.array([[0,0,0,2],
              [0,0,0,0],
              [0,0,0,0],
              [2,0,0,0]])

verifier(A, T)

In Algebra A = [2]
T is Schur Idempotent:

[[0 0 0 2]
 [0 0 0 0]
 [0 0 0 0]
 [2 0 0 0]]
Laplacian:
[[ 2.  0.  0. -2.]
 [ 0.  2.  0.  0.]
 [ 0.  0.  2.  0.]
 [-2.  0.  0.  2.]]
Laplacian Eigenvalues:
[4. 0. 2. 2.]
Degree Matrix:
[[2. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 2.]]
Laplacian Rank:
3
Trace/2:
4.0


In [ ]:
# Computes complete graph in A.
J = A.CompleteGraph()

# Verify its propierties
verifier(A, J)

In Algebra A = [3]
T is Schur Idempotent:

[[3. 0. 0. 0. 3. 0. 0. 0. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 3. 0. 0. 0. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 3. 0. 0. 0. 3.]]
Laplacian:
[[ 6.  0.  0.  0. -3.  0.  0.  0. -3.]
 [ 0.  9.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  9.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  9.  0.  0.  0.  0.  0.]
 [-3.  0.  0.  0.  6.  0.  0.  0. -3.]
 [ 0.  0.  0.  0.  0.  9.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  9.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  9.  0.]
 [-3.  0.  0.  0. -3.  0.  0.  0.  6.]]
Laplacian Eigenvalues:
[ 9. -0.  9.  9.  9.  9.  9.  9.  9.]
Degree Matrix:
[[9. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 9. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 9. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 9. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 9. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 9. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 9. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [0. 0.

In [ ]:
# Compute identity as $mm^t/delta^2$

I = A.m.dot(A.mt).toarray()/9

# Verify its propierties
verifier(A, I)

In Algebra A = [3]
T is Schur Idempotent:

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Laplacian:
[[-0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -0.]]
Laplacian Eigenvalues:
[-0. -0. -0. -0. -0. -0. -0. -0. -0.]
Degree Matrix:
[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0.

# 4. Construction for Quantum Graphs

## 4.1 The 9-Payley Quantum graph

This is a verification of the properties of the quantum 9-Paley Graph. Reader may verify the spectrum coincides with the classical spectrum of 9-Paley graph.

In [ ]:
A = Algebra([2], trace_norm)


A.m.dot(A.mt).toarray()

array([[4., 0., 0., 0.],
       [0., 4., 0., 0.],
       [0., 0., 4., 0.],
       [0., 0., 0., 4.]])

In [ ]:
A = Algebra([3], trace_norm)

G = np.array([[ 2.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.],
                [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
                [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
                [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
                [-1.,  0.,  0.,  0.,  2.,  0.,  0.,  0., -1.],
                [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
                [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
                [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
                [-1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  2.]])

# Is idempotent up to a renormalization
1/3* G.dot(G)

array([[ 2.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
       [-1.,  0.,  0.,  0.,  2.,  0.,  0.,  0., -1.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
       [-1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  2.]])

In [ ]:
# Is a Schur Idempotent
P9 = Rotate(G, 3) #Rotates the projector to get adjacency
verifier(A, P9)

In Algebra A = [3]
T is Schur Idempotent:

[[ 2.  0.  0.  0.  1.  0.  0.  0.  1.]
 [ 0. -1.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  1. -1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  2.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0. -1.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  1. -1.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0. -1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  2.]]
Laplacian:
[[ 2.  0.  0.  0. -1.  0.  0.  0. -1.]
 [ 0.  5.  0.  0.  0. -1. -1.  0.  0.]
 [ 0.  0.  5. -1.  0.  0.  0. -1.  0.]
 [ 0.  0. -1.  5.  0.  0.  0. -1.  0.]
 [-1.  0.  0.  0.  2.  0.  0.  0. -1.]
 [ 0. -1.  0.  0.  0.  5. -1.  0.  0.]
 [ 0. -1.  0.  0.  0. -1.  5.  0.  0.]
 [ 0.  0. -1. -1.  0.  0.  0.  5.  0.]
 [-1.  0.  0.  0. -1.  0.  0.  0.  2.]]
Laplacian Eigenvalues:
[ 3. -0.  6.  3.  6.  3.  3.  6.  6.]
Degree Matrix:
[[4. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 4. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 4. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 4. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 4. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.

Also Satifies equation:

$P_9^2 = k Id + \lambda P_9 + \mu (J - Id - P_9)$ with $k = 5$, $\lambda = 0$, $\mu = 2$.  

In [ ]:
k = 4
lam = 1
mu = 2

print(np.allclose(np.zeros([9,9]), P9.dot(P9) - (k * np.identity(9) + lam * P9 + mu * (A.CompleteGraph() - np.identity(9) - P9))))

True


## 4.2 An example of a quantum str. reg. graph with no classical analog

In [ ]:
G4=np.array([[0. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1.5],
            [0. , 0. , 0. , 1.5, 0. , 0. , 0. , 0. , 0. ],
            [0. , 0. , 0. , 0. , 0. , 0. , 1.5, 0. , 0. ],
            [0. , 1.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
            [1.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1.5],
            [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1.5, 0. ],
            [0. , 0. , 1.5, 0. , 0. , 0. , 0. , 0. , 0. ],
            [0. , 0. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. ],
            [1.5, 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 0. ]])

verifier(A, G4)

In Algebra A = [3]
T is Schur Idempotent:

[[0.  0.  0.  0.  1.5 0.  0.  0.  1.5]
 [0.  0.  0.  1.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  1.5 0.  0. ]
 [0.  1.5 0.  0.  0.  0.  0.  0.  0. ]
 [1.5 0.  0.  0.  0.  0.  0.  0.  1.5]
 [0.  0.  0.  0.  0.  0.  0.  1.5 0. ]
 [0.  0.  1.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  1.5 0.  0.  0. ]
 [1.5 0.  0.  0.  1.5 0.  0.  0.  0. ]]
Laplacian:
[[ 3.   0.   0.   0.  -1.5  0.   0.   0.  -1.5]
 [ 0.   3.   0.  -1.5  0.   0.   0.   0.   0. ]
 [ 0.   0.   3.   0.   0.   0.  -1.5  0.   0. ]
 [ 0.  -1.5  0.   3.   0.   0.   0.   0.   0. ]
 [-1.5  0.   0.   0.   3.   0.   0.   0.  -1.5]
 [ 0.   0.   0.   0.   0.   3.   0.  -1.5  0. ]
 [ 0.   0.  -1.5  0.   0.   0.   3.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -1.5  0.   3.   0. ]
 [-1.5  0.   0.   0.  -1.5  0.   0.   0.   3. ]]
Laplacian Eigenvalues:
[4.5 0.  4.5 1.5 4.5 1.5 4.5 1.5 4.5]
Degree Matrix:
[[3. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 3. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 0. 0. 0. 0. 0.

Also Satifies equation:

$P_9^2 = k Id + \lambda P_9 + \mu (J - Id - P_9)$ with $k = 3$, $\lambda = 0.75$, $\mu = 0.75$.  

In [ ]:
k = 3
lam = 0.75
mu = 0.75

print(np.allclose(np.zeros([9,9]), G4.dot(G4) - (k * np.identity(9) + lam * G4 + mu * (A.CompleteGraph() - np.identity(9) - G4))))

True


## 4.3 The 16-Clebsch graph

This is a verification of the properties of the quantum 16-Clebsh Graph

In [ ]:
A = Algebra([4], trace_norm)

A.say_hi()

Algebra 



<IPython.core.display.Math object>

Dimension = 16
State operator: 
tr = [0.25 0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.25 0.   0.   0.
 0.   0.25]
Multiplication operator: 


<IPython.core.display.Math object>

<IPython.core.display.Math object>

[[16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16. 

16-Clebsch matrix is the cocyle twist of the Cayley graph:
$Cl(16) = Cay(\mathbb{Z}_2^4, \{ \{(1,0,0,0), (0,1,0,0), (0,0,1,0), (0,0,0,1), (1,1,1,1)\} \})$.

In [ ]:
K = np.array([[ 2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.],
       [ 0.,  0.,  0., -2.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
         0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0., -2.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -2.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -2.,
         0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  2.]])

#valores, vectores=np.linalg.eig(K)
#print(valores)
#Kp = Rotate(K,4)
#verifier(A, K)


This new graph has the same minimal polynomial as Classical 16-Clebsch:

$(x+3)^3(x-1)^{10}(x-5)$

In [ ]:
P = np.linalg.matrix_power(K+3*np.identity(16), 3) @ np.linalg.matrix_power(K - np.identity(16),10) @ (K-5*np.identity(16))

print(P)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Also Satifies equation:

$K^2 = k Id + \lambda K + \mu (J - Id - K)$ with $k = 5$, $\lambda = 0$, $\mu = 2$.  

In [ ]:
k = 5
lam = 0
mu = 2

K.dot(K) - (k * np.identity(16) + lam * K + mu * (A.CompleteGraph() - np.identity(16) - K))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

Old proposal for 16-Clebsch. This is in fact Q-isomorphic to the Cayley Graph of $\mathbb{Z}_2^4$ with generators $\{e_i\}$. Is NOT strongly regular.



In [ ]:
A = Algebra([4], trace_norm)

G = np.array([[ 2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 0.,  0.,  0.],
              [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., 0.,  0.,  0.],
              [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0., 0.,  0.,  0.],
              [ 0.,  0.,  0., -2.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0., 0.,  0.,  0.],
              [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.,  1.,  0.],
              [ 1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.,  0.,  1.],
              [ 0.,  0.,  0.,  1.,  0.,  0., -2.,  0.,  0.,  0.,  0.,  0.,  1., 0.,  0.,  0.],
              [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.,  0.,  0.],
              [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.,  0.,  0.],
              [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -2.,  0.,  0.,  1., 0.,  0.,  0.],
              [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0., 0.,  0.,  1.],
              [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.,  1.,  0.],
              [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -2., 0.,  0.,  0.],
              [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0., 0.,  0.,  0.],
              [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., 0.,  0.,  0.],
              [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 0.,  0.,  2.]])

verifier(A, G)

In Algebra A = [4]
T is Schur Idempotent:

[[ 2.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -2.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.  0. -2.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0. -2.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0. -2.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  

## 4.4 Shrinkhande Graph

Shrinkhande matrix is the cocyle twist of the Cayley graph:
$B = Cay(\mathbb{Z}_4^2, \{ \{\pm(1,0), \pm(0,1), \pm(1,1)\} \})$.

In [ ]:
A = Algebra([4], trace_norm)

A.say_hi()

Algebra 



<IPython.core.display.Math object>

Dimension = 16
State operator: 
tr = [0.25 0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.25 0.   0.   0.
 0.   0.25]
Multiplication operator: 


<IPython.core.display.Math object>

<IPython.core.display.Math object>

[[16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16. 

In [ ]:
B = np.array([[ 2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  2.-0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  2.+0.j],
       [ 0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+1.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,  1.-1.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
        -0.-0.j,  0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.-1.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         1.+1.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+1.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  1.-1.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.-0.j,  0.+0.j],
       [ 2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  2.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.-1.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+1.j, -0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
        -2.+0.j, -0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.-0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j, -2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.-0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.-0.j,  1.+1.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         1.-1.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  2.-0.j],
       [ 0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.-1.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,  1.+1.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+1.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.-1.j, -0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -2.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.-1.j,  0.-0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  1.+1.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 2.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  2.+0.j]])


verifier(A,B)

In Algebra A = [4]
T is Schur Idempotent:

[[ 2.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  2.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  2.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+1.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  1.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -2.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+1.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   1.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 2.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  2.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  2.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  1.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j 

This new graph has the same minimal polynomial as Classical 16-Clebsch:

$(x+2)^9(x-2)^{6}(x-6)$

In [ ]:
P = np.linalg.matrix_power(B+2*np.identity(16), 9) @ np.linalg.matrix_power(B - 2* np.identity(16),6) @ (B-6*np.identity(16))

print(np.all(P == np.zeros([16,16], dtype=complex)))

True


Also Satifies equation:

$B^2 = k Id + \lambda B + \mu (J - Id - B)$ with $k = 6$, $\lambda = 2$, $\mu = 2$.  

In [ ]:
k = 6
lam = 2
mu = 2

R = B.dot(B) - (k * np.identity(16) + lam * B + mu * (A.CompleteGraph() - np.identity(16) - B))

print(np.all(R == np.zeros([16,16], dtype=complex)))

True


In [ ]:
A.SchurProd(B, np.identity(16))

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.

# 5. Verifications for Higher Regularity

In [ ]:
A = Algebra([3], trace_norm)

id=np.identity(A.dim())

G2=np.array([[3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3]])
##El grafo completo tiene que ser reflexivo.
#Reflexivo
# Es tres regular con parámetros q_0=6, q_1=q_2=q_3=0. k=8, lambda=7, mu=0.
#Check3PR(A,G2,G2, 9, 8, 7, 0, 6, 0, 0, 0) #Dice que es 3-regular :)))))
print(regularity(A, G2)) #Es conexo, laplaciano (traza, rango)=(72,8).

4.014409254246384e-13
Is regular, degree = 9.0
Is connected
Trace of Lap = 72.0
Rank of Lap = 8
(True, True, np.float64(9.0))


In [ ]:
J = np.ones([9,9])

# for T in Regs:
#   if T[1] == 9:
#     print("Adjancency: ")
#     print(T[0])
#     print("Laplacian: ")
#     L = A.laplacian(T[0])
#     print(L)
#     print(np.linalg.eig(L)[0])
#     print()
#     print(T[0].dot(J))
#     print("\n")


In [ ]:
#This must be quantum isomorphic to the classical complete tripartite graph K_{3,3,3}.
#G3 three is the biprojection onto the diagonal matrices inside M_3.

G3=np.array([[3, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 3, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 3]])
#Reflexivo
#verifier(A,G3)
#G3=G3-id
#two_Gromada(A,G3, G2, 9) #G3 es 2-regular con parámetros k=2, lambda=1, mu=0
#print(Parameters3PR(A,G3, G2, 9)) #q0=q1=q2=q3=0.
#Check3PR(A,G3, G2, 9, 2, 1, 0, 0, 0, 0, 0) #Dice que es 3-regular :)))))
#print(regularity(A, G3)) #No es conexo, laplaciano (traza, rango)=(18, 6)
valores, vectores=np.linalg.eig(G3)
print(valores)

[3. 0. 0. 0. 3. 0. 0. 0. 3.]


In [ ]:
G4=np.array([[1. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1.5],
            [0. , 1. , 0. , 1.5, 0. , 0. , 0. , 0. , 0. ],
            [0. , 0. , 1. , 0. , 0. , 0. , 1.5, 0. , 0. ],
            [0. , 1.5, 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
            [1.5, 0. , 0. , 0. , 1. , 0. , 0. , 0. , 1.5],
            [0. , 0. , 0. , 0. , 0. , 1. , 0. , 1.5, 0. ],
            [0. , 0. , 1.5, 0. , 0. , 0. , 1. , 0. , 0. ],
            [0. , 0. , 0. , 0. , 0. , 1.5, 0. , 1. , 0. ],
            [1.5, 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1. ]])
#Reflexivo
#valores, vectores=np.linalg.eig(G4)
#print(valores)
#print(vectores)
two_Gromada(A,G4-np.identity(9), G2, 9)
#G4 es 2-regular con parámetros k=3, lambda=0.75, mu=0.75
# Es 3-regular con parámetros q_2=q_0=3/8 y q_1=q_3=-3/8.
print(Parameters3PR(A,G4, G2, 9))
Check3PR(A,G4-np.identity(9),G2, 9, 3, 0.75, 0.75, 0.375, -0.375, 0.375, -0.375) #. Da que es 3-regular :)))))
#print(regularity(G4,A)) # Es conexo, laplaciano (traza, rango)=(27, 8)

1.7832822334594722e-15
The matrix is 2-regular. Their parameters are: k=2.9999999999999996, lambda=0.7499999999999999, mu=0.7499999999999999.
(np.float64(0.37500000000000006), np.float64(-0.37499999999999994), np.float64(0.3749999999999999), np.float64(-0.3749999999999999))
1.2469658095878717e-13


np.True_

In [ ]:
G4=np.array([[1. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1.5],
            [0. , 1. , 0. , 1.5, 0. , 0. , 0. , 0. , 0. ],
            [0. , 0. , 1. , 0. , 0. , 0. , 1.5, 0. , 0. ],
            [0. , 1.5, 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
            [1.5, 0. , 0. , 0. , 1. , 0. , 0. , 0. , 1.5],
            [0. , 0. , 0. , 0. , 0. , 1. , 0. , 1.5, 0. ],
            [0. , 0. , 1.5, 0. , 0. , 0. , 1. , 0. , 0. ],
            [0. , 0. , 0. , 0. , 0. , 1.5, 0. , 1. , 0. ],
            [1.5, 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1. ]])
#Reflexivo
valores, vectores=np.linalg.eig(G4)
#print(valores)
print(vectores)
#two_Gromada(A,G4, G2, 9)
#G4 es 2-regular con parámetros k=3, lambda=0.75, mu=0.75
# Es 3-regular con parámetros q_2=q_0=3/8 y q_1=q_3=-3/8.
#print(Parameters3PR(A,G4, G2, 9))
#Check3PR(A,G4,G2, 9, 3, 0.75, 0.75, 0.375, -0.375, 0.375, -0.375) #. Da que es 3-regular :)))))
#print(regularity(G4,A)) # Es conexo, laplaciano (traza, rango)=(27, 8)

G5=np.array([[3/2. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1.5],
            [0. , 0. , 0. , 1.5, 0. , 0. , 0. , 0. , 0. ],
            [0. , 0. , 3/2. , 0. , 0. , 0. , 1.5, 0. , 0. ],
            [0. , 1.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
            [1.5, 0. , 0. , 0. , 3. , 0. , 0. , 0. , 1.5],
            [0. , 0. , 0. , 0. , 0. , 0. , 0. , 1.5, 0. ],
            [0. , 0. , 1.5, 0. , 0. , 0. , 3/2. , 0. , 0. ],
            [0. , 0. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. ],
            [1.5, 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 3/2. ]])

#G5 no es nada XD.
#TwoRegGromada(A,G5, G2, 9) #G5 no es 2-regular.
#print(regularity(G5,A)) #Es conexa, laplaciano (traza, rango)=(36,8)

Q9P = np.array([[ 2.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
              [ 0.,  -1.,  0.,  0.,  0., 1.,  1.,  0.,  0.],
              [ 0.,  0.,  -1., 1.,  0.,  0.,  0.,  1.,  0.],
              [ 0.,  0.,  1.,  -1.,  0.,  0.,  0., 1.,  0.],
              [ 1.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  1.],
              [ 0.,  1.,  0.,  0.,  0.,  -1., 1.,  0.,  0.],
              [ 0., 1.,  0.,  0.,  0.,  1.,  -1.,  0.,  0.],
              [ 0.,  0., 1.,  1.,  0.,  0.,  0.,  -1.,  0.],
              [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.]])
#Irreflexivo
#Q9P=Q9P+id
#valores, vectores=np.linalg.eig(Q9P)
#print(valores)
#two_Gromada(A,Q9P, G2, 9) #k=4, lambda=1, mu=2.
#print(constantes(A,Q9P, G2, 9)) #q_0=0, q_1=0, q_2=1, q_3=0.
#check3pointRegularity(A,Q9P, G2, 9, 4, 1, 2, 0, 0, 1, 0) #Dice que es 3-regular :)))))
#print(regularity(Q9P,A)) #Es conexo, laplaciano (traza, rango)=(36, 8)


G6 = np.array([[ 3. ,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  1.5],
              [ 0. ,  0. ,  0. , -1.5,  0. ,  0. ,  0. ,  0. ,  0. ],
              [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1.5,  0. ,  0. ],
              [ 0. , -1.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
              [ 1.5,  0. ,  0. ,  0. ,  3. ,  0. ,  0. ,  0. ,  1.5],
              [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1.5,  0. ],
              [ 0. ,  0. , -1.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
              [ 0. ,  0. ,  0. ,  0. ,  0. , -1.5,  0. ,  0. ,  0. ],
              [ 1.5,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  3. ]])
#Reflexivo
#valores, vectores=np.linalg.eig(G6)
#print(valores)
#two_Gromada(A,G6, G2, 9) #k=5, lambda=1.75, mu=3.75
#print(constantes(A,G6,G2,9)) #q0=-0.375, q1=0.875, q2=2.625, q3=1.875
#check3pointRegularity(A,G6,G2, 9, 5, 1.75, 3.75, -0.375, 0.875, 2.625, 1.875) #Da que es 3-regular :)))))
#print(regularity(G6,A)) #Es conexo, laplaciano (traza, rango)=(45, 8)



G7=np.array([[1., 0., 0., 0., 3., 0., 0., 0., 3.],
            [0., 1., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 1., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 1., 0., 0., 0., 0., 0.],
            [3., 0., 0., 0., 1., 0., 0., 0., 3.],
            [0., 0., 0., 0., 0., 1., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 1., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0.],
            [3., 0., 0., 0., 3., 0., 0., 0., 1.]])
#Reflexivo
#valores, vectores=np.linalg.eig(G7)
#print(valores)
#TwoRegGromada(A,G7, G2, 9) #k=6, lambda=3, mu=6
#print(Parameters3PR(A,G7,G2,9))#q_0=0, q_1=3, q_2=0, q_3=6. El q_2 está libre, lo que significa que está XD.
#Check3PR(A,G7,G2, 9, 6, 3, 6, 0, 3, 0, 6) #Dice que es 3-regular :)))))
#print(regularity(G7,A)) #Es conexo, laplaciano (traza, rango)=(54, 8)

G8 =np.array([[1., 0., 0., 0., 0., 0., 0., 0., 3.],
              [0., 1., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 1., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 1., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 1., 0., 0., 0., 3.],
              [0., 0., 0., 0., 0., 1., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 1., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 1., 0.],
              [3., 0., 0., 0., 3., 0., 0., 0., 1.]])
#Reflexivo
#print(regularity(G8,A)) #Es conexo, laplaciano (traza, rango)=(36, 8)
#No es ni regular.

#B = Algebra([4])
#B.get_trace(Trace)
#B.get_mult_mat()
K = np.array([[ 2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.],
       [ 0.,  0.,  0., -2.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
         0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0., -2.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -2.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -2.,
         0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  2.]])

Comp = np.array([[ 4.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,
         0.,  0.,  4.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 4.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,
         0.,  0.,  4.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 4.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,
         0.,  0.,  4.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 0.,  0.],
        [ 4.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,
         0.,  0.,  4.]])
#Comp.
#TwoRegGromada(B, K, Comp, 16)
#Check3PR(B, K, Comp, 16, 5, 0, 2, -1000, 0, 0, 1)
#regularity(K,B)

[[-8.16496581e-01  5.77350269e-01 -1.57009246e-16  2.02138599e-02
   4.27325041e-17  1.65062337e-02 -3.96803450e-03 -5.04035495e-18
   3.15898879e-03]
 [ 0.00000000e+00  0.00000000e+00  7.07106781e-01  7.06890055e-01
  -1.92450090e-01  5.77232280e-01 -1.38764399e-01  2.26997407e-02
   6.90447598e-02]
 [ 0.00000000e+00  0.00000000e+00  7.40148683e-17 -7.39921830e-17
   6.80413817e-01  4.08164860e-01 -9.81212479e-02 -8.02557027e-02
  -7.32330267e-02]
 [ 0.00000000e+00  0.00000000e+00  7.07106781e-01 -7.06890055e-01
  -1.92450090e-01 -5.77232280e-01  1.38764399e-01  2.26997407e-02
  -6.90447598e-02]
 [ 4.08248290e-01  5.77350269e-01 -7.40148683e-17 -1.01069299e-02
   6.23730591e-17 -8.25311684e-03  6.46134998e-01  6.14251390e-17
   2.47365148e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.37011323e-01  7.02170737e-01
  -6.54132116e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   6.80413817e-01 -4.0816486

In [ ]:
import numpy as np

n = 4
omega = np.exp(2j * np.pi / n)  # i

# función para índice lineal
def idx(i,j):
    return i*n + j

# función lambda_{a,b}
def lambda_ab(a,b):
    return (omega**a + omega**(-a)) + (omega**b + omega**(-b)) + (omega**(a-b) + omega**(-(a-b)))

# construir la matriz tilde_A
Atil = np.zeros((n*n, n*n), dtype=complex)

for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                if (i-j) % n == (k-l) % n:  # delta_{i-j, k-l} mod n
                    b = (i-j) % n
                    s = 0+0j
                    for a in range(n):
                        s += omega**(a*((i-k)%n)) * lambda_ab(a,b)
                    Atil[idx(i,j), idx(k,l)] = s / n

Atil


array([[ 2.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  2.00000000e+00-4.44089210e-16j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         2.22044605e-16+3.67394040e-16j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  2.00000000e+00+1.66533454e-16j],
       [ 0.00000000e+00+0.00000000e+00j, -6.10622664e-16+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         1.00000000e+00+1.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+0

In [ ]:
def iterate_schur(G, algebra, tol=1e-10, max_iter=100):
    """
    Iteratively find a Schur-idempotent starting from A
    """
    X = G.copy()
    for n in range(max_iter):
        X_new = algebra.SchurProd(X, X)/algebra.dim()**2
        if np.linalg.norm(X_new - X) < tol:
            break
        X = X_new
    return X


# 6. Graph Spectrums

In this section we sumarize all the graphs, known to be strongly regular, and we compute its laplacian and its spectrums. Reader may verify that, in the cases for quantum analogs of classical graphs, the spectrums coincide.

In [ ]:
A = Algebra([2], trace_norm)

In [ ]:
T = np.array([[1,0,0,0],
               [0,1,0,0],
               [0,0,1,0],
               [0,0,0,1]])

print(f"Graph =\n {T}")
print(f"Spectrum = {np.linalg.eig(T)[0]}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0])}")

Graph =
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
Spectrum = [1. 1. 1. 1.]
Laplacian =
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Laplacian spectrum = [0. 0. 0. 0.]


In [ ]:
T = np.array([[2,0,0,0],
               [0,0,0,0],
               [0,0,0,0],
               [0,0,0,2]])

print(f"Graph =\n {T}")
print(f"Spectrum = {np.linalg.eig(T)[0]}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0])}")

Graph =
 [[2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 2]]
Spectrum = [2. 0. 0. 2.]
Laplacian =
 [[0. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 0.]]
Laplacian spectrum = [0. 2. 2. 0.]


In [ ]:
T = np.array([[1,0,0,2],
               [0,1,0,0],
               [0,0,1,0],
               [2,0,0,1]])

print(f"Graph =\n {T}")
print(f"Spectrum = {np.round(np.linalg.eig(T)[0])}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0])}")

Graph =
 [[1 0 0 2]
 [0 1 0 0]
 [0 0 1 0]
 [2 0 0 1]]
Spectrum = [ 3. -1.  1.  1.]
Laplacian =
 [[ 2.  0.  0. -2.]
 [ 0.  2.  0.  0.]
 [ 0.  0.  2.  0.]
 [-2.  0.  0.  2.]]
Laplacian spectrum = [4. 0. 2. 2.]


In [ ]:
T = np.array([[2,0,0,2],
               [0,0,0,0],
               [0,0,0,0],
               [2,0,0,2]])

print(f"Graph =\n {T}")
print(f"Spectrum = {np.round(np.linalg.eig(T)[0])}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0])}")

Graph =
 [[2 0 0 2]
 [0 0 0 0]
 [0 0 0 0]
 [2 0 0 2]]
Spectrum = [4. 0. 0. 0.]
Laplacian =
 [[ 2.  0.  0. -2.]
 [ 0.  4.  0.  0.]
 [ 0.  0.  4.  0.]
 [-2.  0.  0.  2.]]
Laplacian spectrum = [4. 0. 4. 4.]


In [ ]:
A = Algebra([3], trace_norm)

In [ ]:
T =np.array([[3, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 3, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 3]])

print(f"Graph =\n {T}")
print(f"Spectrum = {np.round(np.linalg.eig(T)[0])}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0])}")

Graph =
 [[3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 3]]
Spectrum = [3. 0. 0. 0. 3. 0. 0. 0. 3.]
Laplacian =
 [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 3. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 3. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Laplacian spectrum = [0. 3. 3. 3. 0. 3. 3. 3. 0.]


In [ ]:
T =np.array([[1. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1.5],
            [0. , 1. , 0. , 1.5, 0. , 0. , 0. , 0. , 0. ],
            [0. , 0. , 1. , 0. , 0. , 0. , 1.5, 0. , 0. ],
            [0. , 1.5, 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
            [1.5, 0. , 0. , 0. , 1. , 0. , 0. , 0. , 1.5],
            [0. , 0. , 0. , 0. , 0. , 1. , 0. , 1.5, 0. ],
            [0. , 0. , 1.5, 0. , 0. , 0. , 1. , 0. , 0. ],
            [0. , 0. , 0. , 0. , 0. , 1.5, 0. , 1. , 0. ],
            [1.5, 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1. ]])

print(f"Graph =\n {T}")
print(f"Spectrum = {np.round(np.linalg.eig(T)[0],3)}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0],3)}")

Graph =
 [[1.  0.  0.  0.  1.5 0.  0.  0.  1.5]
 [0.  1.  0.  1.5 0.  0.  0.  0.  0. ]
 [0.  0.  1.  0.  0.  0.  1.5 0.  0. ]
 [0.  1.5 0.  1.  0.  0.  0.  0.  0. ]
 [1.5 0.  0.  0.  1.  0.  0.  0.  1.5]
 [0.  0.  0.  0.  0.  1.  0.  1.5 0. ]
 [0.  0.  1.5 0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.  0.  1.5 0.  1.  0. ]
 [1.5 0.  0.  0.  1.5 0.  0.  0.  1. ]]
Spectrum = [-0.5  4.   2.5 -0.5  2.5 -0.5 -0.5  2.5 -0.5]
Laplacian =
 [[ 3.  0.  0.  0. -2.  0.  0.  0. -2.]
 [ 0.  3.  0. -2.  0.  0.  0.  0.  0.]
 [ 0.  0.  3.  0.  0.  0. -2.  0.  0.]
 [ 0. -2.  0.  3.  0.  0.  0.  0.  0.]
 [-2.  0.  0.  0.  3.  0.  0.  0. -2.]
 [ 0.  0.  0.  0.  0.  3.  0. -2.  0.]
 [ 0.  0. -2.  0.  0.  0.  3.  0.  0.]
 [ 0.  0.  0.  0.  0. -2.  0.  3.  0.]
 [-2.  0.  0.  0. -2.  0.  0.  0.  3.]]
Laplacian spectrum = [4.5 0.  4.5 1.5 4.5 1.5 4.5 1.5 4.5]


In [ ]:
T = np.array([[ 3. ,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  1.5],
              [ 0. ,  0. ,  0. , -1.5,  0. ,  0. ,  0. ,  0. ,  0. ],
              [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1.5,  0. ,  0. ],
              [ 0. , -1.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
              [ 1.5,  0. ,  0. ,  0. ,  3. ,  0. ,  0. ,  0. ,  1.5],
              [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -1.5,  0. ],
              [ 0. ,  0. , -1.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
              [ 0. ,  0. ,  0. ,  0. ,  0. , -1.5,  0. ,  0. ,  0. ],
              [ 1.5,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  3. ]])

print(f"Graph =\n {T}")
print(f"Spectrum = {np.round(np.linalg.eig(T)[0], 3)}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0],3)}")

Graph =
 [[ 3.   0.   0.   0.   1.5  0.   0.   0.   1.5]
 [ 0.   0.   0.  -1.5  0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.  -1.5  0.   0. ]
 [ 0.  -1.5  0.   0.   0.   0.   0.   0.   0. ]
 [ 1.5  0.   0.   0.   3.   0.   0.   0.   1.5]
 [ 0.   0.   0.   0.   0.   0.   0.  -1.5  0. ]
 [ 0.   0.  -1.5  0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -1.5  0.   0.   0. ]
 [ 1.5  0.   0.   0.   1.5  0.   0.   0.   3. ]]
Spectrum = [ 1.5  6.  -1.5 -1.5  1.5  1.5  1.5  1.5 -1.5]
Laplacian =
 [[ 3.  0.  0.  0. -2.  0.  0.  0. -2.]
 [ 0.  6.  0.  2.  0.  0.  0.  0.  0.]
 [ 0.  0.  6.  0.  0.  0.  2.  0.  0.]
 [ 0.  2.  0.  6.  0.  0.  0.  0.  0.]
 [-2.  0.  0.  0.  3.  0.  0.  0. -2.]
 [ 0.  0.  0.  0.  0.  6.  0.  2.  0.]
 [ 0.  0.  2.  0.  0.  0.  6.  0.  0.]
 [ 0.  0.  0.  0.  0.  2.  0.  6.  0.]
 [-2.  0.  0.  0. -2.  0.  0.  0.  3.]]
Laplacian spectrum = [4.5 0.  4.5 7.5 4.5 7.5 4.5 4.5 7.5]


In [ ]:
Tg = np.array([[ 2.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1.],
                [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
                [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
                [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
                [-1.,  0.,  0.,  0.,  2.,  0.,  0.,  0., -1.],
                [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
                [ 0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
                [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.],
                [-1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  2.]])

T = Rotate(Tg, 3)

print(f"Graph =\n {T}")
print(f"Spectrum = {np.round(np.linalg.eig(T)[0])}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0])}")

Graph =
 [[ 2.  0.  0.  0.  1.  0.  0.  0.  1.]
 [ 0. -1.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  1. -1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  2.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0. -1.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  1. -1.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0. -1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  2.]]
Spectrum = [ 1.  4. -2. -2.  1.  1.  1. -2. -2.]
Laplacian =
 [[ 2.  0.  0.  0. -1.  0.  0.  0. -1.]
 [ 0.  5.  0.  0.  0. -1. -1.  0.  0.]
 [ 0.  0.  5. -1.  0.  0.  0. -1.  0.]
 [ 0.  0. -1.  5.  0.  0.  0. -1.  0.]
 [-1.  0.  0.  0.  2.  0.  0.  0. -1.]
 [ 0. -1.  0.  0.  0.  5. -1.  0.  0.]
 [ 0. -1.  0.  0.  0. -1.  5.  0.  0.]
 [ 0.  0. -1. -1.  0.  0.  0.  5.  0.]
 [-1.  0.  0.  0. -1.  0.  0.  0.  2.]]
Laplacian spectrum = [ 3. -0.  6.  3.  6.  3.  3.  6.  6.]


In [ ]:
A = Algebra([4], trace_norm)

In [ ]:
T = np.array([[ 2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.],
       [ 0.,  0.,  0., -2.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
         0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0., -2.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -2.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -2.,
         0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  2.]])

print(f"Graph =\n {T}")
print(f"Spectrum = {np.round(np.linalg.eig(T)[0])}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0])}")

Graph =
 [[ 2.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0. -2.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  2.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.  0. -2.  0.  0.  1.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  1.  0.  0. -2.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  2.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0. -2.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.

In [ ]:
T = np.array([[ 2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  2.-0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  2.+0.j],
       [ 0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+1.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,  1.-1.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
        -0.-0.j,  0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.-1.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         1.+1.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+1.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  1.-1.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.-0.j,  0.+0.j],
       [ 2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  2.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.-1.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+1.j, -0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
        -2.+0.j, -0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.-0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j, -2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.-0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.-0.j,  1.+1.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         1.-1.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  2.-0.j],
       [ 0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.-1.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,  1.+1.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+1.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.-1.j, -0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j, -2.+0.j,
         0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.-1.j,  0.-0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  1.+1.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j],
       [ 2.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  0.+0.j,  0.+0.j,  2.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j,  2.+0.j]])

print(f"Graph =\n {T}")
print(f"Spectrum = {np.round(np.linalg.eig(T)[0])}")
print(f"Laplacian =\n {np.round(A.Laplacian(T))}")
print(f"Laplacian spectrum = {np.round(np.linalg.eig(A.Laplacian(T))[0])}")

Graph =
 [[ 2.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  2.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  2.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+1.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  1.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -2.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+1.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   1.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 2.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  2.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  2.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  1.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -2

# 7. Self Duality Schemes

## 7.1 Star-Triangle / Reidemeister III verifier and Kauffman Exchange relation

In [ ]:
def StarTriangle(A, d, Wplus, Wminus): ### Star-Triangle Equation: (Wplus \otimes Wminus) \circ (comultiplication) \circ (Wplus) == d (Rotate (Wminus)) \circ (Wplus \otimes Wminus) \circ (comultiplication)
  delta = A.trace.dot(A.counit)[0]

  LHS = Wplus.dot( A.m.toarray().dot(np.kron(Wplus, Wminus) ))    ## left hand side of (QSM5)
  RHS = d * A.m.toarray().dot((np.kron(Wplus, Wminus)).dot((np.kron(A.m.toarray(), np.identity(A.dim))).dot((np.kron(np.identity(A.dim), np.kron(Wminus, np.identity(A.dim)))).dot(np.kron(np.identity(A.dim), A.mt.toarray())))))


  if np.allclose(LHS, RHS):
    print("d, Wplus, Wminus DO satisfy the Star-Triangle Equations:\n")
    print(LHS)
    print()
    print(RHS)
  else:
    print("d, Wplus, Wminus DO NOT satisfy the Star-Triangle Equations:\n")
    print(LHS)
    print()
    print(RHS)
    return None


  return None


def StarTriangleVerifier(alg, W_plus, W_minus, d, tol=1e-10):
    n = alg.dim()
    # convert m and mt to arrays
    m_arr = np.asarray(alg.m.toarray())   # (n, n^2)
    mt_arr = np.asarray(alg.mt.toarray()) # (n^2, n)

    assert W_plus.shape == (n, n), "W_plus debe ser n x n"
    assert W_minus.shape == (n, n), "W_minus debe ser n x n"
    assert m_arr.shape == (n, n**2), f"m tiene forma {m_arr.shape}, esperar ({n},{n**2})"
    assert mt_arr.shape == (n**2, n), f"mt tiene forma {mt_arr.shape}, esperar ({n**2},{n})"

    I_n = np.eye(n)
    Wp_kron_Wm = np.kron(W_plus, W_minus)        # (n^2, n^2)

    LHS_mat = W_plus @ m_arr @ Wp_kron_Wm        # (n, n^2)

    A1 = np.kron(I_n, mt_arr)                    # (n^3, n^2)

    A2 = np.kron(I_n, np.kron(W_minus, I_n))     # (n^3, n^3)

    A3 = np.kron(m_arr, I_n)                     # (n^2, n^3)
    prod_mid = A3 @ (A2 @ A1)                     # (n^2, n^2)

    RHS_mat = d * m_arr @ (Wp_kron_Wm @ prod_mid) # (n, n^2)

    # residuo y decisión
    resid = np.linalg.norm(LHS_mat - RHS_mat, ord='fro')
    if resid <= tol:
      return True
    else:
      return False

In [ ]:
def KauffmanExchangeVer(A, d, Wplus, Wminus, eps, tol=1e-10):

    # Construct both sides of the equation
    LHS = Wplus + eps * Wminus
    RHS = (d * np.identity(A.dim()) + A.CompleteGraph())    ### Creo que A.CompleteGraph() debe reescalarse por np.sqrt(A.dim()) para que de 'matriz de 1'

    # Flatten matrices for elementwise comparison
    LHS_flat = LHS.flatten()
    RHS_flat = RHS.flatten()

    # Consider only positions where RHS ≠ 0
    mask = np.abs(RHS_flat) > tol
    if not np.any(mask):
        # If RHS ≈ 0, it can only be a multiple if LHS ≈ 0 as well
        if np.all(np.abs(LHS_flat) < tol):
            return True, 0.0
        else:
            return False, None

    # Compute elementwise ratios where RHS ≠ 0
    ratios = LHS_flat[mask] / RHS_flat[mask]

    # Check if all ratios are approximately equal (within tolerance)
    if np.all(np.abs(ratios - ratios[0]) < tol):
        t = ratios[0]
        return True, t
    else:
        return False, None

## 7.2 Association Scheme data for A3, the quantum square

Building the irreflexive A3 quantum square and its irreflexive complement

In [ ]:
A = Algebra([2], trace_norm)
#A.say_hi()
A3=np.array([[1,0,0,2],
              [0,1,0,0],
              [0,0,1,0],
              [2,0,0,1]])/4  ### Adjacency for quantum square on M_2

KM2=A.CompleteGraph()/4 ### Adjacency for complete graph on M_2

irrKM2 = KM2 - np.identity(4)/4
print(A.SchurProd(irrKM2, np.identity(4)))

#valores, vectores=np.linalg.eig(A3)
#print(valores)
#print(vectores)
#two_Gromada(A,A3, K2, 4)
#A3 es 2-regular con parámetros k=3, lambda=0.75, mu=0.75
# Es 3-regular con parámetros q_2=q_0=3/8 y q_1=q_3=-3/8.
#print(Parameters3PR(A,A3, K2, 4))
#Check3PR(A,A3,K2, 4, 3, 0.75, 0.75, 0.375, -0.375, 0.375, -0.375) #. Da que es 3-regular :)))))
#print(regularity(G4,A)) # Es conexo, laplaciano (traza, rango)=(27, 8)


iA3 = A3 - np.identity(4)/4 # Turns G$ into irreflexive graph
#valores, vectores=np.linalg.eig(iA3)
#print(valores)
#print(vectores)
#two_Gromada(A,iA3, K2, 4)
#A3 es 2-regular con parámetros k=3, lambda=0.75, mu=0.75
# Es 3-regular con parámetros q_2=q_0=3/8 y q_1=q_3=-3/8.
#print(Parameters3PR(A,iA3, K2, 4))
#Check3PR(A,A3,K2, 4, 3, 0.75, 0.75, 0.375, -0.375, 0.375, -0.375) #. Da que es 3-regular :)))))
#print(regularity(G4,A)) # Es conexo, laplaciano (traza, rango)=(27, 8)


[[ 4.26642159e-17  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  4.26642159e-17  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.26642159e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.26642159e-17]]


Some testing...

In [ ]:
print('A3')
print(A3)
print(A.SchurProd(A3, A3))

print('iA3')
iA3 = A3 - np.identity(4)/4
print(iA3)
print(A.SchurProd(iA3, iA3))

print('triv')
print(A.SchurProd(np.identity(4)/4, np.identity(4)/4))

print('KM2')
print(KM2, '\n')
print(A.SchurProd(KM2, KM2))

print('iKM2')
print(irrKM2)
print(A.SchurProd(irrKM2, irrKM2))


A3
[[0.25 0.   0.   0.5 ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.5  0.   0.   0.25]]
[[0.25 0.   0.   0.5 ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.5  0.   0.   0.25]]
iA3
[[0.  0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0. ]]
[[0.  0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0. ]]
triv
[[0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25]]
KM2
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]] 

[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]
iKM2
[[ 0.25  0.    0.    0.5 ]
 [ 0.   -0.25  0.    0.  ]
 [ 0.    0.   -0.25  0.  ]
 [ 0.5   0.    0.    0.25]]
[[ 0.25  0.    0.    0.5 ]
 [ 0.   -0.25  0.    0.  ]
 [ 0.    0.   -0.25  0.  ]
 [ 0.5   0.    0.    0.25]]


Self-duality data and constructions of Boltzman Wplus, Wminus for quantum square

In [ ]:
PA3 = np.array(([1, 2, 1],
                [1, 0, -1],
                [1, -2, 1]))
print(PA3@PA3)  ##Verifying self-duality

#From Jaeger 3.6.2:
epsilon = -1 # fixed value for free sign parameter
t= 3        # fixed value for free nonzero parameter
Wplus = np.array([[0, 0, 0, 2*epsilon*t],
                  [0, -2/t, 0, 0],
                  [0, 0, -2/t, 0],
                  [2*epsilon*t, 0, 0, 0]])

Wminus = np.array([[0, 0, 0, 2*epsilon/t],
                   [0, -2*t, 0, 0],
                   [0, 0, -2*t, 0],
                   [2*epsilon/t, 0, 0, 0]])



[[4 0 0]
 [0 4 0]
 [0 0 4]]


Verifying QSM1-4 for quantum spin model of quantum square

In [ ]:
#LHS = Wplus.dot((A.m.toarray()).dot(np.kron(Wplus, Wminus)))    ## left hand side of (QSM5)
#print(LHS, '\n')
#RHS = d * A.m.toarray().dot((np.kron(Wplus, Wminus)).dot((np.kron(A.m.toarray(), np.identity(4))).dot((np.kron(np.identity(4), np.kron(Wminus, np.identity(4)))).dot(np.kron(np.identity(4), A.mt.toarray()))))) #@ np.kron(I, np.kron(Wminus, I)) @ (np.kron(I), A.mt)
d = -2*epsilon
J = KM2*4
Id = np.identity(4)

####
# QSM1 id \star Wplus = a id, and id \star Wminus = 1/a id
print('QSM1: Wplus')
print(np.around((A.SchurProd(Id, Wplus))/4,2) )
print('QSM1: Wminus')
print(np.around((A.SchurProd(Id, Wminus))/4,2) )  #QSM1 checks for quantum square (notice added the normalization in convolution product by \delta^2=4)
print()

# QSM2 J Wplus = d/a J and J Wminus = da J
print('QSM2 Wplus:')
print ( np.around((J @ Wplus).real,4))
print ( np.around((J @ Wplus).imag,4))
#
print('QSM2 Wminus:')
print ( np.around((J @ Wminus).real,4))
print ( np.around((J @ Wminus).imag,4))        # QSM2checks for quantum square
print()

# QSM3 Wplus \star Wminus = J
print('QSM3:')
print(np.around((A.SchurProd(Wplus, Wminus)/4).real,1))
print(np.around((A.SchurProd(Wplus, Wminus)/4).imag,1))     #QSM3 checks for quantum square (notice added the normalization in convolution product by \delta^2=4)
print()

# QSM4 Wplus \circ Wminus = d^2 id
print('QSM4:')
print(np.around(np.dot(Wplus, Wminus).real,1))
print(np.around(np.dot(Wplus,Wminus).imag,1))             #QSM1 checks for quantum square (notice added the normalization in convolution product by \delta^2=4)

QSM1: Wplus
[[-0.33  0.    0.    0.  ]
 [ 0.   -0.33  0.    0.  ]
 [ 0.    0.   -0.33  0.  ]
 [ 0.    0.    0.   -0.33]]
QSM1: Wminus
[[-3.  0.  0.  0.]
 [ 0. -3.  0.  0.]
 [ 0.  0. -3.  0.]
 [ 0.  0.  0. -3.]]

QSM2 Wplus:
[[-12.   0.   0. -12.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [-12.   0.   0. -12.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
QSM2 Wminus:
[[-1.3333  0.      0.     -1.3333]
 [ 0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.    ]
 [-1.3333  0.      0.     -1.3333]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

QSM3:
[[2. 0. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 2.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

QSM4:
[[4. 0. 0. 0.]
 [0. 4. 0. 0.]
 [0. 0. 4. 0.]
 [0. 0. 0. 4.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Verifying QSM5 for quantum square

In [ ]:
#StarTriangleVerifier(A, Wplus, Wminus, d)
n = A.dim()
# convert m and mt to arrays
m_arr = np.asarray(A.m.toarray())   # (n, n^2)
mt_arr = np.asarray(A.mt.toarray()) # (n^2, n)
I_n = np.eye(n)


Wp_kron_Wm = np.kron(Wplus, Wminus)        # (n^2, n^2)
LHS_mat = d* Wplus @ m_arr @ Wp_kron_Wm        # (n, n^2)
#print(np.around(LHS_mat,0))

A1 = np.kron(I_n, mt_arr)                    # (n^3, n^2)
A2 = np.kron(I_n, np.kron(Wminus, I_n))     # (n^3, n^3)
A3 = np.kron(m_arr, I_n)                     # (n^2, n^3)
prod_mid = A3 @ (A2 @ A1)                     # (n^2, n^2)


RHS_mat =  m_arr @ (Wp_kron_Wm @ prod_mid) # (n, n^2)

print(np.around(LHS_mat - RHS_mat,2).real)
print(np.around(LHS_mat -RHS_mat,2).imag)   ###   Thus the constructed Wplus Wminus spin model for quantum square satisfy QSM5


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Verifying the Kauffman exchange relation for the quantum square

In [ ]:
z = t+ epsilon/t
print('z=', np.round(z,3))
LHS = Wplus + epsilon * Wminus
RHS = z*(d * Id + epsilon*J)
print('real(LHS-RHS)=', '\n', np.around(LHS.real - RHS.real,2))
print('Imag(LHS-RHS)=', '\n', np.around(LHS.imag - RHS.imag,2))

KauffmanExchangeVer(A, d, Wplus, Wminus, epsilon)       ## Therefore, the quantum square spin model givesevaluations of Kauffman Polynomial with z = 8/3


z= 2.667
real(LHS-RHS)= 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Imag(LHS-RHS)= 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


(False, None)

Some knot evaluations for the quantum square

In [ ]:
## Trivial knot id, s_1
print('Trivial knot')
print(np.trace(np.identity(4))/4)
print(np.trace(Wplus)/4)
print(np.trace(Wminus)/4)
print()

## Trefoil knot: s_1^3
#### Jones poly: -q^{-4} + q^{-3} + q^{-1}
print('+Trefoil knot')
print(np.trace(Wplus@Wplus@Wplus)/16)
print(-t**(-4)+t**(-3)+t**(-1) )
print('-Trefoil knot')
print(np.trace(Wminus@Wminus@Wminus)/16)
print(-t**(4)+t**(3)+t**(1) )
print()

## Figure Eight-knot: s_1 s_2^{-1}s_1 s_2^{-1}
## https://katlas.org/wiki/4_1
### Jones Poly: q^2 + q^{-2} - q - q^{-1} + 1
print('Figure Eight knot')
print()

## Hopf link: s_1^2
print('Hopf link')
print(np.trace(Wplus@Wplus)/16)
print(np.trace(Wminus@Wminus)/16)
print()

Trivial knot
1.0
-0.3333333333333333
-3.0

+Trefoil knot
-0.037037037037037035
0.35802469135802467
-Trefoil knot
-27.0
-51

Figure Eight knot

Hopf link
4.555555555555555
4.555555555555555



Boltzmann weights for the classical square

In [ ]:
clWplus = np.array([[-1/t, epsilon*t, 1/t, epsilon*t],
                  [epsilon*t, -1/t, epsilon*t, 1/t],
                  [1/t, epsilon*t, -1/t, epsilon*t],
                  [epsilon*t, 1/t, epsilon*t, -1/t]])

clWminus = np.array([[-t, epsilon/t, t, epsilon/t],
                  [epsilon/t, -t, epsilon/t, t],
                  [t, epsilon/t, -t, epsilon/t],
                  [epsilon/t, t, epsilon/t, -t]])


Some knot evaluations for the classical square

In [ ]:
## Trivial knot id, s_1
print('Trivial knot')
print(np.trace(np.identity(4))/4)
print(np.trace(clWplus)/4)
print(np.trace(clWminus)/4)
print()

## Trefoil knot: s_1^3
#### Jones poly: -q^{-4} + q^{-3} + q^{-1}
print('+Trefoil knot')
print(np.trace(clWplus@clWplus@clWplus)/16)
print(-t**(-4)+t**(-3)+t**(-1) )
print('-Trefoil knot')
print(np.trace(clWminus@clWminus@clWminus)/16)
print(-t**(4)+t**(3)+t**(1) )
print()

## Figure Eight-knot: s_1 s_2^{-1}s_1 s_2^{-1}
## https://katlas.org/wiki/4_1
### Jones Poly: q^2 + q^{-2} - q - q^{-1} + 1
print('Figure Eight knot')
print()

## Hopf link: s_1^2
print('Hopf link')
print(np.trace(clWplus@clWplus)/16)
print(np.trace(clWminus@clWminus)/16)
print()

Trivial knot
1.0
-0.3333333333333333
-3.0

+Trefoil knot
-0.037037037037036945
0.35802469135802467
-Trefoil knot
-27.0
-51

Figure Eight knot

Hopf link
4.555555555555555
4.555555555555555



## 7.3 Association Scheme data for G3


Setting up G3 and its irreflexive complement

In [ ]:
A = Algebra([3], trace_norm)
#A.say_hi()
#This must be quantum isomorphic to the classical complete tripartite graph K_{3,3,3}.
#G3 three is the biprojection onto the diagonal matrices inside M_3.

G3=np.array([[3, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 3, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 3]])
#Reflexivo
#verifier(A,G3)
#G3=G3-id
#two_Gromada(A,G3, G2, 9) #G3 es 2-regular con parámetros k=2, lambda=1, mu=0
#print(Parameters3PR(A,G3, G2, 9)) #q0=q1=q2=q3=0.
#Check3PR(A,G3, G2, 9, 2, 1, 0, 0, 0, 0, 0) #Dice que es 3-regular :)))))
#print(regularity(A, G3)) #No es conexo, laplaciano (traza, rango)=(18, 6)

KM3=np.array([[3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3]])
iG3 = IrreGraph(A, G3)
iG3comp = IrreGraph(A, KM3 - iG3)

print('Spectrum G3')
valores, vectores=np.linalg.eig(G3)
print(valores)

print('Spectrum iG3')
valores, vectores=np.linalg.eig(iG3)
print(valores)

print('Spectrum iG3comp')
valores, vectores=np.linalg.eig(iG3comp)
print(valores)

Spectrum G3
[3. 0. 0. 0. 3. 0. 0. 0. 3.]
Spectrum iG3
[ 2. -1. -1. -1.  2. -1. -1. -1.  2.]
Spectrum iG3comp
[-3.  6. -3.  0.  0.  0.  0.  0.  0.]


Self-duality data for G3

In [ ]:
PG3=np.array([[1, 6, 2],
              [1, 0,-1],
              [1,-3,2]]) ## by columns, P = [ [1,1,1], [k, s, r], [n-k-1, -s-1, -r-1]  ]
print(PG3 @ PG3)
np.linalg.det(PG3)    ## Does not give a self-dual association scheme. Another problem is that its (G3, G3comp) is not a connected pair


iG3comp = IrreGraph(A,KM3-iG3)

[[9 0 0]
 [0 9 0]
 [0 0 9]]


Built of spin model for G3


In [ ]:
### Using Section 3.4 of Jaeger, STRONGLY REGULAR GRAPHS AND SPIN MODELS FOR THE KAUFFMAN POLYNOMIAL
epsilon = -1 #\epsilon \in {+1, -1}
s = 0
r = -3

d = epsilon * (r - s)# d = \epsilon (r - s)
tsquared = complex(-3, np.sqrt(9-4*epsilon))/(2*epsilon)
t = np.sqrt(tsquared)

a = -s*epsilon*t + (r+1)/t

print('d=',d,'t=',t, 'tsquared= ', tsquared, 'a=',a)
Wplus  = a*np.identity(9) + epsilon*t*iG3 + iG3comp/t
Wminus = np.identity(9)/a + epsilon*iG3/t + t*iG3comp


#print(Wplus.real)
#print(Wplus.imag)

print('Wplus @ Wplus')
print((Wplus @ Wplus).real)
print((Wplus @ Wplus).imag)

print('Wplus star Wplus')
print(np.around((A.SchurProd(Wplus, Wplus)/9).real),0)
print(np.around((A.SchurProd(Wplus, Wplus)/9).imag),0)


d= 3 t= (1.3865799435863255-0.6500799488954088j) tsquared=  (1.5-1.8027756377319946j) a= (-1.182479349027706-0.5543900431716791j)
Wplus @ Wplus
[[ 20.           0.           0.           0.         -12.81818182
    0.           0.           0.         -12.81818182]
 [  0.          -1.40909091   0.           0.           0.
    0.           0.           0.           0.        ]
 [  0.           0.          -1.40909091   0.           0.
    0.           0.           0.           0.        ]
 [  0.           0.           0.          -1.40909091   0.
    0.           0.           0.           0.        ]
 [-12.81818182   0.           0.           0.          20.
    0.           0.           0.         -12.81818182]
 [  0.           0.           0.           0.           0.
   -1.40909091   0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
    0.          -1.40909091   0.           0.        ]
 [  0.           0.           0.           0.     

Verifying QSM 1-4 for G3

In [ ]:
J = KM3
Id = np.identity(9)


# QSM1 id \star Wplus = a id, and id \star Wminus = 1/a id
print('QSM1: Wplus', '\n')
print(np.around((A.SchurProd(Id, Wplus)/9),1) )
print('QSM1: Wminus', '\n')
print(np.around((A.SchurProd(Id, Wminus)/9),1) )  #QSM1 checks for G4

# QSM2 J Wplus = d/a J and J Wminus = da J
print('QSM2:', '\n')
print ( np.around((J @ Wplus).real,1))
print ( np.around((J @ Wplus).imag,1))
#
print ( np.around((J @ Wminus).real,1))
print ( np.around((J @ Wminus).imag,1))        # QSM2 checks for G4


# QSM3 Wplus \star Wminus = J
print('QSM3:', '\n')
print(np.around((A.SchurProd(Wplus, Wminus)/9).real,1))
print(np.around((A.SchurProd(Wplus, Wminus)/9).imag,1))     #QSM3 checks for G4


# QSM4 Wplus \circ Wminus = d^2 id
print('QSM4:', '\n')
print(np.around(np.dot(Wplus, Wminus).real,1))
print(np.around(np.dot(Wplus,Wminus).imag,1))             #QSM4 Checks for (epsilon = -1, t = =1, -1) and (epsilon =1, t = +i, -i)

QSM1: Wplus 

[[-1.2-0.6j  0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j  -1.2-0.6j  0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j  -1.2-0.6j  0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j  -1.2-0.6j  0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j  -1.2-0.6j  0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -1.2-0.6j  0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -1.2-0.6j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
  -1.2-0.6j  0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j  -1.2-0.6j]]
QSM1: Wminus 

[[-0.7+0.3j  0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j  -0.7+0.3j  0. +0.

## 7.4 Association scheme data for G4

Setting up G4 graph and its complement

In [ ]:
A = Algebra([3], trace_norm)
#A.say_hi()

G4=np.array([[1. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1.5],
            [0. , 1. , 0. , 1.5, 0. , 0. , 0. , 0. , 0. ],
            [0. , 0. , 1. , 0. , 0. , 0. , 1.5, 0. , 0. ],
            [0. , 1.5, 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
            [1.5, 0. , 0. , 0. , 1. , 0. , 0. , 0. , 1.5],
            [0. , 0. , 0. , 0. , 0. , 1. , 0. , 1.5, 0. ],
            [0. , 0. , 1.5, 0. , 0. , 0. , 1. , 0. , 0. ],
            [0. , 0. , 0. , 0. , 0. , 1.5, 0. , 1. , 0. ],
            [1.5, 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1. ]])
#Reflexive
#valores, vectores=np.linalg.eig(G4)
#print(valores)
#print(vectores)
#two_Gromada(A,G4, G2, 9)
#G4 es 2-regular con parámetros k=3, lambda=0.75, mu=0.75
# Es 3-regular con parámetros q_2=q_0=3/8 y q_1=q_3=-3/8.
#print(Parameters3PR(A,G4, G2, 9))
#Check3PR(A,G4,G2, 9, 3, 0.75, 0.75, 0.375, -0.375, 0.375, -0.375) #. Da que es 3-regular :)))))
#print(regularity(G4,A)) # Es conexo, laplaciano (traza, rango)=(27, 8)
KM3=np.array([[3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3]])

#print(regularity(A, G2))

iG4=IrreGraph(A, G4) # Turns G$ into irreflexive graph
#verifier(A, iG4)
valores, vectores=np.linalg.eig(iG4) ## we organize the eigenvalues as k = 3 > s = 1.g \geq 0 \geq r = -1.5
print(valores)
print()
#print(vectores)
#two_Gromada(A,iG4, G2, 9)   ### iG4 is 2-point regular
#Check3PR(A,iG4,G2, 9, 3, 0.75, 0.75, 0.375, -0.375, 0.375, -0.375) ### iG4 is 3-point regular
#LiG4 = A.Laplacian(iG4)
#print(regularity(A,iG4))

iEdge4=Rotate(iG4,3)
print(iG4)

[-1.5  3.   1.5 -1.5 -1.5  1.5 -1.5  1.5 -1.5]

[[0.  0.  0.  0.  1.5 0.  0.  0.  1.5]
 [0.  0.  0.  1.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  1.5 0.  0. ]
 [0.  1.5 0.  0.  0.  0.  0.  0.  0. ]
 [1.5 0.  0.  0.  0.  0.  0.  0.  1.5]
 [0.  0.  0.  0.  0.  0.  0.  1.5 0. ]
 [0.  0.  1.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  1.5 0.  0.  0. ]
 [1.5 0.  0.  0.  1.5 0.  0.  0.  0. ]]


Irreflexive complement of G4 and its edge projection

In [ ]:
iG4comp = IrreGraph(A,KM3-iG4)
print(iG4comp)

#verifier(A, iG4comp)
valores, vectores=np.linalg.eig(iG4comp)
print('adjacency eigenvalues',valores)
print()
#print(vectores)
#two_Gromada(A,iG4comp, G2, 9)   ### iG4comp is 2-point regular
#Check3PR(A,iG4comp,G2, 9, 5, 14/8, 30/8, -3/8, 7/8, 21/8, 15/8) ### iG4comp is 3-point regular
#LiG4comp = A.Laplacian(iG4comp)
#print(regularity(A,iG4comp))

#iEdge4comp=Rotate(iG4comp,3)
#print(iEdge4comp)

[[ 2.   0.   0.   0.   1.5  0.   0.   0.   1.5]
 [ 0.  -1.   0.  -1.5  0.   0.   0.   0.   0. ]
 [ 0.   0.  -1.   0.   0.   0.  -1.5  0.   0. ]
 [ 0.  -1.5  0.  -1.   0.   0.   0.   0.   0. ]
 [ 1.5  0.   0.   0.   2.   0.   0.   0.   1.5]
 [ 0.   0.   0.   0.   0.  -1.   0.  -1.5  0. ]
 [ 0.   0.  -1.5  0.   0.   0.  -1.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -1.5  0.  -1.   0. ]
 [ 1.5  0.   0.   0.   1.5  0.   0.   0.   2. ]]
adjacency eigenvalues [ 0.5  5.   0.5 -2.5 -2.5  0.5  0.5 -2.5  0.5]



Self-duality data for G4

In [ ]:
### We now check the duality equation P^2 = nI, where P is the matrix of the self-duality (Fourier) wrt {E_i}, the orthogonal basis of \circ-idempotents of the Bose-Mesner algebra
PG4=np.array([[1,  3,   5],
            [1, 3/2, -5/2],
            [1, -3/2, 1/2]]) ## by columns, P = [ [1,1,1], [k, s, r], [n-k-1, -s-1, -r-1]  ]


PG4prime=np.array([[1,  3,   5],
            [1, -3/2, 1/2],
            [1, 3/2, -5/2]]) #Exchanging r and s from PG4

valores, vectores=np.linalg.eig(PG4)
print('adjacency eigenvalues',valores)

print(PG4 @PG4) ## =9I, verifying the association scheme is self dual
print()

valores, vectores=np.linalg.eig(PG4prime)
print('adjacency eigenvalues',valores)

np.dot(PG4prime, PG4prime) ## not self dual

## Thus, for G4 we must choose s = 3/2 and r = -3/2

adjacency eigenvalues [ 3. -3.  3.]
[[9. 0. 0.]
 [0. 9. 0.]
 [0. 0. 9.]]

adjacency eigenvalues [ 3. -3. -3.]


array([[ 9.,  6., -6.],
       [ 0.,  6.,  3.],
       [ 0., -3., 12.]])

Built of spin model for G4

In [ ]:
### Using Section 3.4 of Jaeger, STRONGLY REGULAR GRAPHS AND SPIN MODELS FOR THE KAUFFMAN POLYNOMIAL
epsilon = 1 #\epsilon \in {+1, -1}
s = 3/2
r = -3/2

d = epsilon * (r - s)# d = \epsilon (r - s)
t = complex(0,-1)  # t = \pm sqrt{-epsilon}, solving for t in [Equation (33), Jaeger]
a = -s*epsilon*t + (r+1)/t

print('d=',d,'t=',t,'a=',a)
Wplus  = a*np.identity(9) + epsilon*t*iG4 + iG4comp/t
Wminus = np.identity(9)/a + epsilon*iG4/t + t*iG4comp

print('Wplus')
print(Wplus.real)
print(Wplus.imag)

print('Wminus')
print(Wminus.real)
print(Wminus.imag)


print('Wplus @ Wplus')
print((Wplus @ Wplus).real)
print((Wplus @ Wplus).imag)

print('Wminus @ Wminus')
print((Wminus @ Wminus).real)
print((Wminus @ Wminus).imag)

print('Wplus star Wplus')
print(np.around((A.SchurProd(Wplus, Wplus)/9).real),0)
print(np.around((A.SchurProd(Wplus, Wplus)/9).imag),0)


d= -3.0 t= -1j a= 1j
Wplus
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -3.  0.  0.]
 [ 0. -3.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  3.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -3.  0.]
 [ 0.  0. -3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -3.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  3.]]
Wminus
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[-3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0.  0.  0.  0.

Verifying QSM1-4 for G4

In [ ]:
J = KM3
Id = np.identity(9)


# QSM1 id \star Wplus = a id, and id \star Wminus = 1/a id
print('QSM1: Wplus', '\n')
print(np.around((A.SchurProd(Id, Wplus)/9),1) )
print('QSM1: Wminus', '\n')
print(np.around((A.SchurProd(Id, Wminus)/9),1) )  #QSM1 checks for G4

# QSM2 J Wplus = d/a J and J Wminus = da J
print('QSM2:', '\n')
print ( np.around((J @ Wplus).real,1))
print ( np.around((J @ Wplus).imag,4))
#
print ( np.around((J @ Wminus).real,4))
print ( np.around((J @ Wminus).imag,4))        # QSM2 checks for G4


# QSM3 Wplus \star Wminus = J
print('QSM3:', '\n')
print(np.around((A.SchurProd(Wplus, Wminus)/9).real,1))
print(np.around((A.SchurProd(Wplus, Wminus)/9).imag,1))     #QSM3 checks for G4


# QSM4 Wplus \circ Wminus = d^2 id
print('QSM4:', '\n')
print(np.around(np.dot(Wplus, Wminus).real,1))
print(np.around(np.dot(Wplus,Wminus).imag,1))             #QSM4 Checks for (epsilon = -1, t = =1, -1) and (epsilon =1, t = +i, -i)

QSM1: Wplus 

[[0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j]]
QSM1: Wminus 

[[0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.

Verifying QSM5 for G4

In [ ]:
#StarTriangleVerifier(A, Wplus, Wminus, d)
n = A.dim()
# convert m and mt to arrays
m_arr = np.asarray(A.m.toarray())   # (n, n^2)
mt_arr = np.asarray(A.mt.toarray()) # (n^2, n)
I_n = np.eye(n)


Wp_kron_Wm = np.kron(Wplus, Wminus)        # (n^2, n^2)
LHS_mat = d* Wplus @ m_arr @ Wp_kron_Wm        # (n, n^2)
#print(np.around(LHS_mat,0))

A1 = np.kron(I_n, mt_arr)                    # (n^3, n^2)
A2 = np.kron(I_n, np.kron(Wminus, I_n))     # (n^3, n^3)
A3 = np.kron(m_arr, I_n)                     # (n^2, n^3)
prod_mid = A3 @ (A2 @ A1)                     # (n^2, n^2)


RHS_mat =  m_arr @ (Wp_kron_Wm @ prod_mid) # (n, n^2)

print(np.around(LHS_mat - RHS_mat,0).real)
print(np.around(LHS_mat -RHS_mat,0).imag)   ###   Thus Wplus Wminus gives spin model for G4 (epsilon = -1, t=+1, -1) (epsilon =1, t = +i,-i)


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.

Verifying Kauffman exchange relation for G4 spin model

In [ ]:
z = t+ epsilon/t
print('z=', np.round(z,2))
LHS = Wplus + epsilon * Wminus
RHS = z*(d * np.identity(A.dim()) + epsilon*J)
print('real(LHS-RHS)=', np.around(LHS.real - RHS.real,2))
print('Imag(LHS-RHS)=', np.around(LHS.imag - RHS.imag,2))

print(KauffmanExchangeVer(A, d, Wplus, Wminus, epsilon) )      ## Therefore, G4 spin model gives degenerate evaluations of Kauffman Polynomial with z = 0

#print(np.around((Wplus + Wminus).real),1)
#print(np.around((Wplus + Wminus).imag),1)     ### In this case, Wplus = - Wminus are Hadamard matrices?

print(Wplus.real)
print(Wplus.imag)

z= 0j
real(LHS-RHS)= [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Imag(LHS-RHS)= [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(True, np.complex128(0j))
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -3.  0.  0.]
 [ 0. -3

## 7.5 Association scheme data for G4

Setting up G4 graph and its complement

In [ ]:
A = Algebra([3], trace_norm)
#A.say_hi()

G4=np.array([[1. , 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1.5],
            [0. , 1. , 0. , 1.5, 0. , 0. , 0. , 0. , 0. ],
            [0. , 0. , 1. , 0. , 0. , 0. , 1.5, 0. , 0. ],
            [0. , 1.5, 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
            [1.5, 0. , 0. , 0. , 1. , 0. , 0. , 0. , 1.5],
            [0. , 0. , 0. , 0. , 0. , 1. , 0. , 1.5, 0. ],
            [0. , 0. , 1.5, 0. , 0. , 0. , 1. , 0. , 0. ],
            [0. , 0. , 0. , 0. , 0. , 1.5, 0. , 1. , 0. ],
            [1.5, 0. , 0. , 0. , 1.5, 0. , 0. , 0. , 1. ]])
#Reflexive
#valores, vectores=np.linalg.eig(G4)
#print(valores)
#print(vectores)
#two_Gromada(A,G4, G2, 9)
#G4 es 2-regular con parámetros k=3, lambda=0.75, mu=0.75
# Es 3-regular con parámetros q_2=q_0=3/8 y q_1=q_3=-3/8.
#print(Parameters3PR(A,G4, G2, 9))
#Check3PR(A,G4,G2, 9, 3, 0.75, 0.75, 0.375, -0.375, 0.375, -0.375) #. Da que es 3-regular :)))))
#print(regularity(G4,A)) # Es conexo, laplaciano (traza, rango)=(27, 8)
KM3=np.array([[3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3]])

#print(regularity(A, G2))

iG4=IrreGraph(A, G4) # Turns G$ into irreflexive graph
#verifier(A, iG4)
valores, vectores=np.linalg.eig(iG4) ## we organize the eigenvalues as k = 3 > s = 1.g \geq 0 \geq r = -1.5
print(valores)
print()
#print(vectores)
#two_Gromada(A,iG4, G2, 9)   ### iG4 is 2-point regular
#Check3PR(A,iG4,G2, 9, 3, 0.75, 0.75, 0.375, -0.375, 0.375, -0.375) ### iG4 is 3-point regular
#LiG4 = A.Laplacian(iG4)
#print(regularity(A,iG4))

iEdge4=Rotate(iG4,3)
print(iG4)

[-1.5  3.   1.5 -1.5 -1.5  1.5 -1.5  1.5 -1.5]

[[0.  0.  0.  0.  1.5 0.  0.  0.  1.5]
 [0.  0.  0.  1.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  1.5 0.  0. ]
 [0.  1.5 0.  0.  0.  0.  0.  0.  0. ]
 [1.5 0.  0.  0.  0.  0.  0.  0.  1.5]
 [0.  0.  0.  0.  0.  0.  0.  1.5 0. ]
 [0.  0.  1.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  1.5 0.  0.  0. ]
 [1.5 0.  0.  0.  1.5 0.  0.  0.  0. ]]


Irreflexive complement of G4 and its edge projection

In [ ]:
iG4comp = IrreGraph(A,KM3-iG4)
print(iG4comp)

#verifier(A, iG4comp)
valores, vectores=np.linalg.eig(iG4comp)
print('adjacency eigenvalues',valores)
print()
#print(vectores)
#two_Gromada(A,iG4comp, G2, 9)   ### iG4comp is 2-point regular
#Check3PR(A,iG4comp,G2, 9, 5, 14/8, 30/8, -3/8, 7/8, 21/8, 15/8) ### iG4comp is 3-point regular
#LiG4comp = A.Laplacian(iG4comp)
#print(regularity(A,iG4comp))

#iEdge4comp=Rotate(iG4comp,3)
#print(iEdge4comp)

[[ 2.   0.   0.   0.   1.5  0.   0.   0.   1.5]
 [ 0.  -1.   0.  -1.5  0.   0.   0.   0.   0. ]
 [ 0.   0.  -1.   0.   0.   0.  -1.5  0.   0. ]
 [ 0.  -1.5  0.  -1.   0.   0.   0.   0.   0. ]
 [ 1.5  0.   0.   0.   2.   0.   0.   0.   1.5]
 [ 0.   0.   0.   0.   0.  -1.   0.  -1.5  0. ]
 [ 0.   0.  -1.5  0.   0.   0.  -1.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -1.5  0.  -1.   0. ]
 [ 1.5  0.   0.   0.   1.5  0.   0.   0.   2. ]]
adjacency eigenvalues [ 0.5  5.   0.5 -2.5 -2.5  0.5  0.5 -2.5  0.5]



Self-duality data for G4

In [ ]:
### We now check the duality equation P^2 = nI, where P is the matrix of the self-duality (Fourier) wrt {E_i}, the orthogonal basis of \circ-idempotents of the Bose-Mesner algebra
PG4=np.array([[1,  3,   5],
            [1, 3/2, -5/2],
            [1, -3/2, 1/2]]) ## by columns, P = [ [1,1,1], [k, s, r], [n-k-1, -s-1, -r-1]  ]


PG4prime=np.array([[1,  3,   5],
            [1, -3/2, 1/2],
            [1, 3/2, -5/2]]) #Exchanging r and s from PG4

valores, vectores=np.linalg.eig(PG4)
print('adjacency eigenvalues',valores)

print(PG4 @PG4) ## =9I, verifying the association scheme is self dual
print()

valores, vectores=np.linalg.eig(PG4prime)
print('adjacency eigenvalues',valores)

np.dot(PG4prime, PG4prime) ## not self dual

## Thus, for G4 we must choose s = 3/2 and r = -3/2

adjacency eigenvalues [ 3. -3.  3.]
[[9. 0. 0.]
 [0. 9. 0.]
 [0. 0. 9.]]

adjacency eigenvalues [ 3. -3. -3.]


array([[ 9.,  6., -6.],
       [ 0.,  6.,  3.],
       [ 0., -3., 12.]])

Built of spin model for G4

In [ ]:
### Using Section 3.4 of Jaeger, STRONGLY REGULAR GRAPHS AND SPIN MODELS FOR THE KAUFFMAN POLYNOMIAL
epsilon = 1 #\epsilon \in {+1, -1}
s = 3/2
r = -3/2

d = epsilon * (r - s)# d = \epsilon (r - s)
t = complex(0,-1)  # t = \pm sqrt{-epsilon}, solving for t in [Equation (33), Jaeger]
a = -s*epsilon*t + (r+1)/t

print('d=',d,'t=',t,'a=',a)
Wplus  = a*np.identity(9) + epsilon*t*iG4 + iG4comp/t
Wminus = np.identity(9)/a + epsilon*iG4/t + t*iG4comp

print('Wplus')
print(Wplus.real)
print(Wplus.imag)

print('Wminus')
print(Wminus.real)
print(Wminus.imag)


print('Wplus @ Wplus')
print((Wplus @ Wplus).real)
print((Wplus @ Wplus).imag)

print('Wminus @ Wminus')
print((Wminus @ Wminus).real)
print((Wminus @ Wminus).imag)

print('Wplus star Wplus')
print(np.around((A.SchurProd(Wplus, Wplus)/9).real),0)
print(np.around((A.SchurProd(Wplus, Wplus)/9).imag),0)


d= -3.0 t= -1j a= 1j
Wplus
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -3.  0.  0.]
 [ 0. -3.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  3.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -3.  0.]
 [ 0.  0. -3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -3.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  3.]]
Wminus
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[-3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0.  0.  0.  0.

Verifying QSM1-4 for G4

In [ ]:
J = KM3
Id = np.identity(9)


# QSM1 id \star Wplus = a id, and id \star Wminus = 1/a id
print('QSM1: Wplus', '\n')
print(np.around((A.SchurProd(Id, Wplus)/9),1) )
print('QSM1: Wminus', '\n')
print(np.around((A.SchurProd(Id, Wminus)/9),1) )  #QSM1 checks for G4

# QSM2 J Wplus = d/a J and J Wminus = da J
print('QSM2:', '\n')
print ( np.around((J @ Wplus).real,1))
print ( np.around((J @ Wplus).imag,4))
#
print ( np.around((J @ Wminus).real,4))
print ( np.around((J @ Wminus).imag,4))        # QSM2 checks for G4


# QSM3 Wplus \star Wminus = J
print('QSM3:', '\n')
print(np.around((A.SchurProd(Wplus, Wminus)/9).real,1))
print(np.around((A.SchurProd(Wplus, Wminus)/9).imag,1))     #QSM3 checks for G4


# QSM4 Wplus \circ Wminus = d^2 id
print('QSM4:', '\n')
print(np.around(np.dot(Wplus, Wminus).real,1))
print(np.around(np.dot(Wplus,Wminus).imag,1))             #QSM4 Checks for (epsilon = -1, t = =1, -1) and (epsilon =1, t = +i, -i)

QSM1: Wplus 

[[0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j]]
QSM1: Wminus 

[[0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.

Verifying QSM5 for G4

In [ ]:
#StarTriangleVerifier(A, Wplus, Wminus, d)
n = A.dim()
# convert m and mt to arrays
m_arr = np.asarray(A.m.toarray())   # (n, n^2)
mt_arr = np.asarray(A.mt.toarray()) # (n^2, n)
I_n = np.eye(n)


Wp_kron_Wm = np.kron(Wplus, Wminus)        # (n^2, n^2)
LHS_mat = d* Wplus @ m_arr @ Wp_kron_Wm        # (n, n^2)
#print(np.around(LHS_mat,0))

A1 = np.kron(I_n, mt_arr)                    # (n^3, n^2)
A2 = np.kron(I_n, np.kron(Wminus, I_n))     # (n^3, n^3)
A3 = np.kron(m_arr, I_n)                     # (n^2, n^3)
prod_mid = A3 @ (A2 @ A1)                     # (n^2, n^2)


RHS_mat =  m_arr @ (Wp_kron_Wm @ prod_mid) # (n, n^2)

print(np.around(LHS_mat - RHS_mat,0).real)
print(np.around(LHS_mat -RHS_mat,0).imag)   ###   Thus Wplus Wminus gives spin model for G4 (epsilon = -1, t=+1, -1) (epsilon =1, t = +i,-i)


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.

Verifying Kauffman exchange relation for G4 spin model

In [ ]:
z = t+ epsilon/t
print('z=', np.round(z,2))
LHS = Wplus + epsilon * Wminus
RHS = z*(d * np.identity(A.dim()) + epsilon*J)
print('real(LHS-RHS)=', np.around(LHS.real - RHS.real,2))
print('Imag(LHS-RHS)=', np.around(LHS.imag - RHS.imag,2))

print(KauffmanExchangeVer(A, d, Wplus, Wminus, epsilon) )      ## Therefore, G4 spin model gives degenerate evaluations of Kauffman Polynomial with z = 0

#print(np.around((Wplus + Wminus).real),1)
#print(np.around((Wplus + Wminus).imag),1)     ### In this case, Wplus = - Wminus are Hadamard matrices?

print(Wplus.real)
print(Wplus.imag)

z= 0j
real(LHS-RHS)= [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Imag(LHS-RHS)= [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(True, np.complex128(0j))
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[ 3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -3.  0.  0.]
 [ 0. -3

## 7.6 Association Scheme Data for 9Paley_q

In [ ]:
A = Algebra([3], trace_norm)  #Tracial 3x3 matrices

Pal = np.array([[ 2.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
                [ 0., -1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.],
                [ 0.,  0., -1.,  1.,  0.,  0.,  0.,  1.,  0.],
                [ 0.,  0.,  1., -1.,  0.,  0.,  0.,  1.,  0.],
                [ 1.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  1.],
                [ 0.,  1.,  0.,  0.,  0., -1.,  1.,  0.,  0.],
                [ 0.,  1.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
                [ 0.,  0.,  1.,  1.,  0.,  0.,  0., -1.,  0.],
                [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.]],)    # quantum 9Paley graph over M3


KM3=np.array([[3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [3, 0, 0, 0, 3, 0, 0, 0, 3]],dtype=int)


iPal=IrreGraph(A, Pal) # Ensuring irreflexivity just in case
#verifier(A, iPal)
#valores, vectores=np.linalg.eig(iPal) ## we organize the eigenvalues as k = 3 > s = 1.g \geq 0 \geq r = -1.5
#print(valores)
#print()
#print(vectores)
#two_Gromada(A,iPal, G2, 9)
# iPal is 2-point with parameters k=4, lambda=1, mu=2
# iPal is 3-point regular with parameters q_3 = 0, q_2 = 0, q_1 = 1, q_0 = 0
# print(Parameters3PR(A,iPal, G2, 9))
# Check3PR(A,iPal,G2, 9, 4, 1, 2, 0, 0, 1, 0) #. Is 3-pt regular :)))))
#print(regularity(iPal,A)) # Is connected, laplaciano (traza, rango)=(36, 4)


NameError: name 'Algebra' is not defined

Some testing

In [ ]:
print('Pal')
print(Pal)
print(A.SchurProd(Pal, Pal)/9)

print('iPal')
#iPal = A3 - np.identity(4)/4
print(iPal)
print(np.around(A.SchurProd(iPal, np.identity(9)),2))

print('triv')
print(np.around(A.SchurProd(np.identity(9), np.identity(9))/9,1))

print('KM3')
print(KM3, '\n')
print(A.SchurProd(KM3, KM3)/9)

#print('iKM3')
#print(irrKM3)
#print(A.SchurProd(irrKM3, irrKM3))

Pal
[[ 2.  0.  0.  0.  1.  0.  0.  0.  1.]
 [ 0. -1.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  1. -1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  2.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0. -1.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  1. -1.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0. -1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  2.]]
[[ 2.  0.  0.  0.  1.  0.  0.  0.  1.]
 [ 0. -1.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  1. -1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  2.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0. -1.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  1. -1.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0. -1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  2.]]
iPal
[[ 2.  0.  0.  0.  1.  0.  0.  0.  1.]
 [ 0. -1.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  1. -1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  2.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0. -1.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  1. -1.  0.  0.]
 [ 0.  0.  1. 

Construction of the complementary graph to iPal

In [ ]:
iPalcomp = IrreGraph(A,KM3-iPal)
#print(iPalcomp)

#verifier(A, iPalcomp)
#valores, vectores=np.linalg.eig(iPalcomp) ## we organize the eigenvalues as k = 3 > s = 1.g \geq 0 \geq r = -1.5
#print(valores)
#print()
#print(vectores)
#two_Gromada(A,iPalcomp, G2, 9)
# iPalcomp is 2-point with parameters k=4, lambda=1, mu=2
print(np.identity(9) + iPal + iPalcomp)

print(A.SchurProd(iPalcomp, iPalcomp)/9)

print('iPalcomp')
print(iPalcomp)

print('iPal')
print(iPal)

[[3. 0. 0. 0. 3. 0. 0. 0. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 3. 0. 0. 0. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 3. 0. 0. 0. 3.]]
[[ 0.  0.  0.  0.  2.  0.  0.  0.  2.]
 [ 0.  0.  0.  0.  0. -1. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0. -1.  0.]
 [ 0.  0. -1.  0.  0.  0.  0. -1.  0.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 0. -1.  0.  0.  0.  0. -1.  0.  0.]
 [ 0. -1.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0. -1. -1.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  2.  0.  0.  0.  0.]]
iPalcomp
[[ 0.  0.  0.  0.  2.  0.  0.  0.  2.]
 [ 0.  0.  0.  0.  0. -1. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0. -1.  0.]
 [ 0.  0. -1.  0.  0.  0.  0. -1.  0.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 0. -1.  0.  0.  0.  0. -1.  0.  0.]
 [ 0. -1.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0. -1. -1.  0.  0.  0.  0.  0.]
 [ 2.  0.  0.  0.  2.  0.  0.  0.  0.]]
iPal
[[ 2.  0.  

Construction of Boltzmann weights Wplus & Wminus for 9Pal_q

In [ ]:
PiPal = np.array(([1,4,4],
                  [1,1,-2],
                  [1,-2,1]))  #Data for self-duality of association scheme
s = 1
r = -2

PiPal2 = np.array(([1,4,4],
                  [1,-2,1],
                  [1,1,-2]))  #Data for self-duality of association scheme which also works


#parameters given by Jaeger Section 3.6.4 The Lattice graphs. For us, q =3 corresponds to 9Paley
epsilon = -1
t = complex(np.sqrt(3)/2, 1/2)  ## There are choices of signs for t.real and t.imag
a = t**3
#a = -s*epsilon*t + (r+1)/t
d = 3

Tplus = np.array(([a],[t], [1/t]))
Tminus = np.array(([1/a],[1/t], [t]))

print(np.around((PiPal @ Tplus - d* Tminus).real),0)
print(np.around((PiPal @ Tplus - d* Tminus).imag),0)   # These values do NOT give solution to PT = dT^- [Equation (16), Jaeger] even using two different definitions for a


print('epsilon =', epsilon, ', t=', np.round(t,2), ', a =', np.round(a,2), ', d=', d)
Wplus = a*np.identity(9) + epsilon*t*iPal + (1/t)*iPalcomp
#Wplus_imag_round = np.around(Wplus.imag, 3)
#print( bmatrix(Wplus.real) + '\n')
#print(bmatrix(np.around(Wplus.imag, 3)))

Wminus = (1/a)*np.identity(9) + epsilon*(1/t)*iPal + t*iPalcomp
#Wminus_imag_round = np.around(Wminus.imag, 3)
#print( bmatrix(Wminus.real) + '\n')
#print(bmatrix(np.around(Wminus.imag, 3)))

### Notice \sqrt{3}  = 1.732, \sqrt{3}/2 = .866 and \sqrt{3}^3/2 = 2.598

[[ 7.]
 [-3.]
 [-3.]] 0
[[ 4.]
 [ 4.]
 [-2.]] 0
epsilon = -1 , t= (0.87+0.5j) , a = (-0+1j) , d= 3


Verifying QSM 1, 2, 3, 4 for quantum 9Paley

In [ ]:
J = KM3
Id = np.identity(9)

# QSM1 id \star Wplus = a id, and id \star Wminus = 1/a id
print('QSM1: Wplus')
print(np.around((A.SchurProd(Id, Wplus)/9),1) )
print('QSM1: Wminus')
print(np.around((A.SchurProd(Id, Wminus)/9),1) )  #QSM1 checks for quantum 9Paley

# QSM2 J Wplus = d/a J and J Wminus = da J
print('QSM2 Wplus:')
print ( np.around((J @ Wplus).real,2))
print ( np.around((J @ Wplus).imag,2))
print('QSM2 Wminus:')
print ( np.around((J @ Wminus).real,2))
print ( np.around((J @ Wminus).imag,2))        # QSM2 checks for quantum 9 Paley


# QSM3 Wplus \star Wminus = J
print('QSM3:', '\n')
print(np.around((A.SchurProd(Wplus, Wminus)/9).real,1))
print(np.around((A.SchurProd(Wplus, Wminus)/9).imag,1))     #QSM3 checks for quantum 9Paley


# QSM4 Wplus \circ Wminus = d^2 id
print('QSM4:', '\n')
print(np.around((Wplus @ Wminus).real,1))
print(np.around((Wplus @ Wminus).imag,1))             #QSM4 checks for quanatum 9Paley

QSM1: Wplus
[[-0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j]]
QSM1: Wminus
[[-0.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -0.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -0.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]

 Verifying QSM5 Star-triangle relation for the quantum 9Paley over M3

In [ ]:
#StarTriangleVerifier(A, Wplus, Wminus, d)
n = A.dim()
# convert m and mt to arrays
m_arr = np.asarray(A.m.toarray())   # (n, n^2)
mt_arr = np.asarray(A.mt.toarray()) # (n^2, n)
I_n = np.eye(n)


Wp_kron_Wm = np.kron(Wplus, Wminus)        # (n^2, n^2)
LHS_mat = d*Wplus @ m_arr @ Wp_kron_Wm        # (n, n^2)
#print(np.around(LHS_mat.real,0))
#print(np.around(LHS_mat.imag,0))

A1 = np.kron(I_n, mt_arr)                    # (n^3, n^2)
A2 = np.kron(I_n, np.kron(Wminus, I_n))     # (n^3, n^3)
A3 = np.kron(m_arr, I_n)                     # (n^2, n^3)
prod_mid = A3 @ (A2 @ A1)                     # (n^2, n^2)


RHS_mat =  m_arr @ (Wp_kron_Wm @ prod_mid) # (n, n^2)

print(np.around(LHS_mat - RHS_mat,0).real)
print(np.around(LHS_mat -RHS_mat,0).imag)   ###   Thus Wplus and Wminus from quantum 9Paley give a quantum spin model



[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0. -0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.  0. -0.
   0.  0. -0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0. -0.  0.  0.
  -0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0. -0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.
   0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0. 

Verifying Kauffman Exchange relation for quantum 9Paley over M3

In [ ]:
z = t+ epsilon/t
print('z=', np.round(z,2))
LHS = Wplus + epsilon * Wminus
RHS = z*(d * Id + epsilon*KM3)
print('real(LHS-RHS)=', np.around(LHS.real - RHS.real,2))
print('Imag(LHS-RHS)=', np.around(LHS.imag - RHS.imag,2))

KauffmanExchangeVer(A, d, Wplus, Wminus, epsilon)       ## Therefore, quantum 9 Paley spin model gives evaluations of Kauffman Polynomial with z = i



z= (-0+1j)
real(LHS-RHS)= [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Imag(LHS-RHS)= [[-0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -0.]]


(False, None)

Evaluations of quantum9Paley spin model on knots and links

In [ ]:
## Trivial knot id, s_1
print('Trivial knot')
print(np.round(np.trace(np.identity(4))/4,2))
print(np.round(np.trace(Wplus)/4,2))
print(np.round(np.trace(Wminus)/4,2))
print()

## Trefoil knot: s_1^3
#### Jones poly: -q^{-4} + q^{-3} + q^{-1}
print('+Trefoil knot')
print(np.round(np.trace(Wplus@Wplus@Wplus)/16,2))
print(np.round(-t**(-4)+t**(-3)+t**(-1),2) )
print('-Trefoil knot')
print(np.round(np.trace(Wminus@Wminus@Wminus)/16,2))
print(np.round(-t**(4)+t**(3)+t**(1),2 ))
print()

## Figure Eight-knot: s_1 s_2^{-1}s_1 s_2^{-1}
## https://katlas.org/wiki/4_1
### Jones Poly: q^2 + q^{-2} - q - q^{-1} + 1
print('Figure Eight knot')
print()

## Hopf link: s_1^2
print('Hopf link')
print(np.trace(Wplus@Wplus)/16)
print(np.trace(Wminus@Wminus)/16)
print()

Trivial knot
1.0
(-0+2.25j)
(-0-2.25j)

+Trefoil knot
(-0+15.19j)
(1.37-0.63j)
-Trefoil knot
(-0-15.19j)
(1.37+0.63j)

Figure Eight knot

Hopf link
(1.6874999999999996-7.216449660063518e-16j)
(1.6874999999999993-4.996003610813204e-16j)



## 7.7 Association Scheme data for classical Paley

In [ ]:
A = Algebra([1,1,1,1,1,1,1,1,1], trace_norm)
clPal = np.array([[0,1,1,1,0,0,1,0,0],
                  [1,0,1,0,1,0,0,1,0],
                  [1,1,0,0,0,1,0,0,1],
                  [1,0,0,0,1,1,1,0,0],
                  [0,1,0,1,0,1,0,1,0],
                  [0,0,1,1,1,0,0,0,1],
                  [1,0,0,1,0,0,0,1,1],
                  [0,1,0,0,1,0,1,0,1],
                  [0,0,1,0,0,1,1,1,0]]) ## Adjacency data for classical 9 Paley

K9=A.CompleteGraph()*3
K9irr=IrreGraph(A,K9)

#verifier(A, clPal)
#valores, vectores=np.linalg.eig(clPal) ## we organize the eigenvalues as k = 3 > s = 1.g \geq 0 \geq r = -1.5
#print(valores)
#print()
#print(vectores)
#two_Gromada(A,clPal, K9, 9)
# clPal is 2-point with parameters k=4, lambda=1, mu=2
# clPal is 3-point regular with parameters q_3 = 0, q_2 = 0, q_1 = 1, q_0 = 0
# print(Parameters3PR(A,clPal, A.CompleteGraph(), 9))
#Check3PR(A,clPal,K9, 9, 4, 1, 2, 0, 0, 1, 0) #. Is 3-pt regular :)))))
#print(regularity(clPal,A))

clPalcomp = IrreGraph(A,K9-clPal) ## The irreflexive complement of cl9Pal
#A.SchurProd(np.identity(9)/3, np.identity(9)/3)
#A.SchurProd(K9irr/3, K9irr/3)

Self-duality matrix P and construction of W for classical 9 Paley

In [ ]:
PiPal = np.array(([1,4,4],
                  [1,1,-2],
                  [1,-2,1]))  #Data for self-duality of association scheme
#print(PiPal @ PiPal)   $ = 9 Id ensures is a duality


#parameters given by Jaeger Section 3.6.4 The Lattice graphs. For us, q =3
epsilon = -1
t = complex(np.sqrt(3)/2, 1/2)  ## There is a choice of sign for t.real and t.imag
a = t**3
d = 3
Wplus = a*np.identity(9) + epsilon*t*clPal + (1/t)*clPalcomp
#Wplus_imag_round = np.around(Wplus.imag, 3)
#print( bmatrix(Wplus.real) + '\n')
#print(bmatrix(np.around(Wplus.imag, 3)))

Wminus = (1/a)*np.identity(9) + epsilon*(1/t)*clPal + t*clPalcomp
#Wminus_imag_round = np.around(Wminus.imag, 3)
#print( bmatrix(Wminus.real) + '\n')
#print(bmatrix(np.around(Wminus.imag, 3)))

### Notice \sqrt{3}  = 1.732, \sqrt{3}/2 = .866 and \sqrt{3}^3/2 = 2.598

print('Wplus.Re','\n' , (np.around(Wplus.real,3)))

print('Wplus.Im','\n' , (np.around(Wplus.imag,3)))

Wplus.Re 
 [[-0.    -0.866 -0.866 -0.866  0.866  0.866 -0.866  0.866  0.866]
 [-0.866 -0.    -0.866  0.866 -0.866  0.866  0.866 -0.866  0.866]
 [-0.866 -0.866 -0.     0.866  0.866 -0.866  0.866  0.866 -0.866]
 [-0.866  0.866  0.866 -0.    -0.866 -0.866 -0.866  0.866  0.866]
 [ 0.866 -0.866  0.866 -0.866 -0.    -0.866  0.866 -0.866  0.866]
 [ 0.866  0.866 -0.866 -0.866 -0.866 -0.     0.866  0.866 -0.866]
 [-0.866  0.866  0.866 -0.866  0.866  0.866 -0.    -0.866 -0.866]
 [ 0.866 -0.866  0.866  0.866 -0.866  0.866 -0.866 -0.    -0.866]
 [ 0.866  0.866 -0.866  0.866  0.866 -0.866 -0.866 -0.866 -0.   ]]
Wplus.Im 
 [[ 1.  -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5]
 [-0.5  1.  -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5  1.  -0.5 -0.5 -0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5  1.  -0.5 -0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5  1.  -0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5 -0.5  1.  -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5 -0.5 -0.5  1.  -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5  1.  -0.5]

Verifying QSM1,2,3,4 for classical 9Paley

In [ ]:
J = np.ones([9,9])
Id = np.identity(9)

# QSM1 id \star Wplus = a id, and id \star Wminus = 1/a id
print('QSM1: Wplus', '\n')
print(np.around((A.SchurProd(Id, Wplus)/9),1) )
print('QSM1: Wminus', '\n')
print(np.around((A.SchurProd(Id, Wminus)/9),1) )  #QSM1 checks for classical 9Paley

# QSM2 J Wplus = d/a J and J Wminus = da J
print('QSM2:', '\n')
print ( np.around((J @ Wplus).real,4))
print ( np.around((J @ Wplus).imag,4))
#
print ( np.around((J @ Wminus).real,4))
print ( np.around((J @ Wminus).imag,4))        #QSM2 checks for classical 9 Paley


# QSM3 Wplus \star Wminus = J
print('QSM3:', '\n')
print(np.around((A.SchurProd(Wplus, Wminus)/9).real,1))
print(np.around((A.SchurProd(Wplus, Wminus)/9).imag,1))     #QSM3 checks for classical 9Paley


# QSM4 Wplus \circ Wminus = d^2 id
print('QSM4:', '\n')
print(np.around(np.dot(Wplus, Wminus).real,1))
print(np.around(np.dot(Wplus,Wminus).imag,1))             #QSM4 Checks for classical 9Paley

QSM1: Wplus 

[[-0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -0.+1.j]]
QSM1: Wminus 

[[-0.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -0.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -0.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -0.-1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+

Verifying Star-Triangle Relation for spin model from classical 9Paley

In [ ]:
#StarTriangleVerifier(A, Wplus, Wminus, d)
n = A.dim()
# convert m and mt to arrays
m_arr = np.asarray(A.m.toarray())   # (n, n^2)
mt_arr = np.asarray(A.mt.toarray()) # (n^2, n)
I_n = np.eye(n)


Wp_kron_Wm = np.kron(Wplus, Wminus)        # (n^2, n^2)
LHS_mat = d* Wplus @ m_arr @ Wp_kron_Wm        # (n, n^2)
#print(np.around(LHS_mat,0))

A1 = np.kron(I_n, mt_arr)                    # (n^3, n^2)
A2 = np.kron(I_n, np.kron(Wminus, I_n))     # (n^3, n^3)
A3 = np.kron(m_arr, I_n)                     # (n^2, n^3)
prod_mid = A3 @ (A2 @ A1)                     # (n^2, n^2)


RHS_mat =  m_arr @ (Wp_kron_Wm @ prod_mid) # (n, n^2)

print(np.around(LHS_mat - RHS_mat,0).real)
print(np.around(LHS_mat -RHS_mat,0).imag)   ###   Thus the constructed Wplus Wminus spin model for classical 9Paley satisfies QSM5


[[ 0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0. -0.
   0.  0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0.
   0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0.
   0.  0.  0.  0. -0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.
   0.  0.  0.  0. -0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0. -0.
   0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.
   0.  0.  0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0. -0.  0. -0.  0.  0.  0.
   0.  0.  0. -0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0.
   0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.
   0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. -0. -0.  0.  0.  0. 

Kauffman exchange relation for cl9Paley

In [ ]:
z = t+ epsilon/t
print('z=', np.round(z,2))
LHS = Wplus + epsilon * Wminus
RHS = z*(d * np.identity(A.dim()) + epsilon*K9)
print('real(LHS-RHS)=', np.around(LHS.real - RHS.real,2))
print('Imag(LHS-RHS)=', np.around(LHS.imag - RHS.imag,2))

KauffmanExchangeVer(A, d, Wplus, Wminus, epsilon)

z= (-0+1j)
real(LHS-RHS)= [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Imag(LHS-RHS)= [[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


(False, None)

Evaluations of classical 9Paley spin model on links & knots


In [ ]:
## Trivial knot id, s_1
print('Trivial knot')
print(np.round(np.trace(np.identity(4))/4,2))
print(np.round(np.trace(Wplus)/4,2))
print(np.round(np.trace(Wminus)/4,2))
print()

## Trefoil knot: s_1^3
#### Jones poly: -q^{-4} + q^{-3} + q^{-1}
print('+Trefoil knot')
print(np.round(np.trace(Wplus@Wplus@Wplus)/16,2))
print(np.round(-t**(-4)+t**(-3)+t**(-1),2) )
print('-Trefoil knot')
print(np.round(np.trace(Wminus@Wminus@Wminus)/16,2))
print(np.round(-t**(4)+t**(3)+t**(1),2 ))
print()

## Figure Eight-knot: s_1 s_2^{-1}s_1 s_2^{-1}
## https://katlas.org/wiki/4_1
### Jones Poly: q^2 + q^{-2} - q - q^{-1} + 1
print('Figure Eight knot')
print()

## Hopf link: s_1^2
print('Hopf link')
print(np.trace(Wplus@Wplus)/16)
print(np.trace(Wminus@Wminus)/16)
print()


Trivial knot
1.0
(-0+2.25j)
(-0-2.25j)

+Trefoil knot
(-0+15.19j)
(1.37-0.63j)
-Trefoil knot
(-0-15.19j)
(1.37+0.63j)

Figure Eight knot

Hopf link
(1.6875-8.1507093862521865e-16j)
(1.6874999999999996-6.586967307034251e-16j)



## 7.8 Association Scheme data for 16Cl_q


Setting up quantum 16 Clebsch graph and its complement

In [ ]:
A = Algebra([4], trace_norm)
#A.say_hi()

Clebsch = np.array([[ 2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.],
       [ 0.,  0.,  0., -2.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
         0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0., -2.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -2.,  0.,  0.,  1.,
         0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,
         0.,  0.,  1.],
       [ 0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0., -2.,
         0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  2.]])                                              ### Irreflexive 16 Clebsch graph

KM4 = A.CompleteGraph()*4

iClebsch=IrreGraph(A, Clebsch) # Ensuring irreflexivity just in case
#verifier(A,iClebsch)
#valores, vectores=np.linalg.eig(iClebsch) ## we organize the eigenvalues as k = 3 > s = 1.g \geq 0 \geq r = -1.5
#print(valores)
#print()
#print(vectores)
#two_Gromada(A,Clebsch, A.CompleteGraph(), 16)
# iClebsch is 2-point with parameters k=5, lambda=0, mu=2

iClebschcomp = IrreGraph(A, A.CompleteGraph() - Clebsch)

#verifier(A, iClebschcomp)

Self-duality data for quantum 16 Clebsch and construction of Wplus and Wminus

In [ ]:
PiClebsch = np.array(([1,5,10],
                  [1,-3,2],
                  [1,1,-2]))  #Data for self-duality of association scheme
print(PiClebsch @ PiClebsch)   # = 16Id ensures is a duality



#parameters given by de La Harpe page 92
epsilon = 1 #\epsilon \in {+1, -1}
r = 1
s = -3
d = epsilon * (r - s)# d = \epsilon (r - s)

t = complex(0,-1)  # t = +-sqrt{-epsilon}
a = -s*epsilon*t + (r+1)/t

print('d=',d,'t=',t,'a=',a)
Wplus  = a*np.identity(16) + epsilon*t*iClebsch + iClebschcomp/t
Wminus = np.identity(16)/a + epsilon*iClebsch/t + t*iClebschcomp


print('Wplus')
print(Wplus.real)
print(Wplus.imag)

print('Wminus')
print(Wminus.real)
print(Wminus.imag)


print('Wplus @ Wplus')
print((Wplus @ Wplus).real)
print((Wplus @ Wplus).imag)

print('Wminus @ Wminus')
print((Wminus @ Wminus).real)
print((Wminus @ Wminus).imag)



print('Wplus star Wplus')
print(np.around((A.SchurProd(Wplus, Wplus)/16).real),0)
print(np.around((A.SchurProd(Wplus, Wplus)/16).imag),0)


##




[[16  0  0]
 [ 0 16  0]
 [ 0  0 16]]
d= 4 t= -1j a= -1j
Wplus
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[-2.  0.  0.  0.  0.  2.  0.  0.  0.  0.  2.  0.  0.  0.  0.  2.]
 [ 0. -2.  0.  0. -2.  0.  0.  0.  0.  0.  0. -2.  0. 

Verifying QSM1-4 for quantum Clebsch

In [ ]:
J = KM4/16
Id = np.identity(16)

# QSM1 id \star Wplus = a id, and id \star Wminus = 1/a id
print('QSM1: Wplus', '\n')
print(np.around((A.SchurProd(Id, Wplus)/16),1) )
print('QSM1: Wminus', '\n')
print(np.around((A.SchurProd(Id, Wminus)/16),1) )  #QSM1 checks for quantum Clebsch

# QSM2 J Wplus = d/a J and J Wminus = da J
print('QSM2:', '\n')
print ( np.around((J @ Wplus).real,4))
print ( np.around((J @ Wplus).imag,4))
#
print ( np.around((J @ Wminus).real,4))
print ( np.around((J @ Wminus).imag,4))        # UP to  factor ???, QSM2 checks for quantum Clebsch


# QSM3 Wplus \star Wminus = J
print('QSM3:', '\n')
print(np.around(A.SchurProd(Wplus, Wminus).real,1))
print(np.around(A.SchurProd(Wplus, Wminus).imag,1))     #Up to factor of ???, QSM3 checks quantum Clebsch


# QSM4 Wplus \circ Wminus = d^2 id
print('QSM4:', '\n')
print(np.around(np.dot(Wplus, Wminus).real,1))
print(np.around(np.dot(Wplus,Wminus).imag,1))             #QSM4 Checks for (epsilon = 1, t = =i, -i) and (epsilon =-1, t = +1, -1)

QSM1: Wplus 

[[0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j

Verifying QSM5 for quantum Clebsch

In [ ]:
#StarTriangleVerifier(A, Wplus, Wminus, d)
n = A.dim()
# convert m and mt to arrays
m_arr = np.asarray(A.m.toarray())   # (n, n^2)
mt_arr = np.asarray(A.mt.toarray()) # (n^2, n)
I_n = np.eye(n)


Wp_kron_Wm = np.kron(Wplus, Wminus)        # (n^2, n^2)
LHS_mat = d* Wplus @ m_arr @ Wp_kron_Wm        # (n, n^2)
#print(np.around(LHS_mat,0))

A1 = np.kron(I_n, mt_arr)                    # (n^3, n^2)
A2 = np.kron(I_n, np.kron(Wminus, I_n))     # (n^3, n^3)
A3 = np.kron(m_arr, I_n)                     # (n^2, n^3)
prod_mid = A3 @ (A2 @ A1)                     # (n^2, n^2)


RHS_mat =  m_arr @ (Wp_kron_Wm @ prod_mid) # (n, n^2)

print(np.around(LHS_mat - RHS_mat,0).real)
print(np.around(LHS_mat -RHS_mat,0).imag)   ###   Thus Wplus Wminus gives spin model for Clebsch (epsilon = 1, t=+i, -i) and (epsilon =-1, t = +1,-1)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Verifying Kauffman exchange relation for quantum Clebsch

In [ ]:
z = t+ epsilon/t
print('z=', np.round(z,5)) # z = 0 for quantum Clebsch

LHS = Wplus + epsilon * Wminus
RHS = z*(d * np.identity(A.dim()) + epsilon*JM4)
print('real(LHS-RHS)=', np.around(LHS.real - RHS.real,2))
print('Imag(LHS-RHS)=', np.around(LHS.imag - RHS.imag,2))

print(KauffmanExchangeVer(A, d, Wplus, Wminus, epsilon) )      ## Therefore, G4 spin model gives degenerate evaluations of Kauffman Polynomial with z = 0

#print(np.around((Wplus + Wminus).real),1)
#print(np.around((Wplus + Wminus).imag),1)     ### In this case, Wplus = - Wminus are Hadamard matrices

print((Wplus + Wminus).real)
print((Wplus + Wminus).imag)

z= 0j


NameError: name 'JM4' is not defined